# PequeShop: Data Preparation Pipeline

## End-to-End Data Science Project with Business Focus

**Methodology:** CRISP-DM + Lean Thinking  
**Data Pipeline:** ETL (Extract, Transform, Load)  
**Focus:** Applied Data Science for E-commerce Analytics

**Author:** Jose Marcel Lopez Pino  
**Date:** February 2026  
**Bootcamp:** Fundamentos de Ciencia de Datos - SENCE/Alkemy

---

### Business Problem

**PequeShop** is a Chilean e-commerce specializing in children's clothing and accessories (ages 4-10). The company started on MercadoLibre (2023), migrated to Shopify (2024), and now promotes through Facebook/Instagram Ads.

**Challenge:** Data is fragmented across multiple platforms with inconsistent formats, missing values, and outliers that prevent unified analytics.

**Business Decision Enabled:** Clean, consolidated data enables Customer Lifetime Value (CLTV) analysis, Customer Acquisition Cost (CAC) optimization, Net Promoter Score (NPS) segmentation, and marketing attribution modeling.

---

### Methodology: CRISP-DM + Lean Thinking

This project follows **CRISP-DM** for structured data science work, combined with **Lean principles** for iterative validation and value generation over excessive complexity.

**Lean principles applied:**
- **Eliminate waste:** Early identification of `customer_id` mapping ensured data traceability across platforms, avoiding rework in later phases.
- **Build-Measure-Learn:** NPS integration was discovered during exploratory analysis and incorporated iteratively, enhancing customer segmentation without delaying the pipeline.
- **Value focus:** Every transformation decision (outlier treatment, feature engineering) was evaluated against business impact, not just technical correctness.

---

### Project Scope: CRISP-DM + Lean

This project covers phases 1-3 of CRISP-DM, applying Lean principles throughout:

| CRISP-DM Phase | Lean Principle Applied | ETL Stage | Lessons |
|----------------|------------------------|-----------|---------|
| ✅ Business Understanding | Value focus | - | Problem definition, KPIs |
| ✅ Data Understanding | Eliminate waste | **Extract** | L1-L3: Early `customer_id` mapping |
| ✅ Data Preparation | Build-Measure-Learn | **Transform** | L4-L5: NPS discovered & integrated iteratively |
| ✅ Data Preparation | Continuous improvement | **Load** | L6: Aggregation, KPIs, export |
| ⏳ Modeling | - | - | *Future: ML models* |
| ⏳ Evaluation | - | - | *Future: Business impact* |
| ⏳ Deployment | - | - | *Future: Dashboard/API* |

**Note:** ETL (Extract → Transform → Load) is the data pipeline pattern used to implement the CRISP-DM phases.

---

### ETL Pipeline Overview

```
EXTRACT                      TRANSFORM                       LOAD
─────────────────────────────────────────────────────────────────────
📄 CSV (MercadoLibre)   →   🔧 Schema harmonization    →   💾 CSV
📊 Excel (Shopify)      →   🚫 Missing value imputation →   📊 Excel
🌐 Web (Marketing)      →   📊 Outlier detection (IQR/Z) 
                        →   ✨ Feature engineering
                        →   🔄 Data wrangling
```

---

### Data Sources

| Source | Format | Description | ETL Phase |
|--------|--------|-------------|----------|
| MercadoLibre | CSV | Historical transactions (2023-2024) | Extract |
| Shopify | Excel | Current platform sales (2024-2025) | Extract |
| Marketing | Web Table | Facebook/Instagram campaign metrics | Extract |

---

## Lesson 1: NumPy - Synthetic Data Generation

**Objective:** Create a fictional dataset of customers and transactions using NumPy arrays, applying basic operations for initial data preparation.

### Why NumPy?

NumPy is efficient for numerical data handling because:

1. **Memory efficiency:** Arrays store elements of the same type contiguously in memory
2. **Vectorized operations:** Operations are applied to entire arrays without explicit loops
3. **Broadcasting:** Automatic handling of arrays with different shapes
4. **C-level performance:** Core operations are implemented in C/Fortran

In [1]:
# Import libraries
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

print(f"NumPy version: {np.__version__}")

NumPy version: 2.4.1


### 1.1 Define Business Parameters

Based on PequeShop's business model:

In [2]:
# =============================================================================
# BUSINESS PARAMETERS
# =============================================================================

# Dataset size
N_CUSTOMERS = 500
N_TRANSACTIONS = 2000

# Product categories and price ranges (CLP)
PRODUCTS = {
    'socks': {'min_price': 2990, 'max_price': 5990},
    'towels': {'min_price': 7990, 'max_price': 15990},
    'tshirts': {'min_price': 8990, 'max_price': 14990},
    'shorts': {'min_price': 9990, 'max_price': 16990},
    'jackets': {'min_price': 19990, 'max_price': 34990},
    'pajamas': {'min_price': 14990, 'max_price': 24990}
}

# Sales platforms
PLATFORMS = ['mercadolibre', 'shopify']

# Acquisition channels
CHANNELS = ['organic', 'mercadolibre_ads', 'google_ads', 'facebook_ads', 'instagram_ads']

# Platform fees
MERCADOLIBRE_FEE = 0.13  # 13% commission
SHOPIFY_PAYMENT_FEE = 0.03  # 3% payment gateway

# Shipping costs (CLP)
SHIPPING_STANDARD = 3500
SHIPPING_EXPRESS = 5500
FREE_SHIPPING_THRESHOLD = 30000

# Chilean regions for customers
REGIONS = [
    'Metropolitana', 'Valparaiso', 'Biobio', 'Araucania', 
    'Maule', 'OHiggins', 'Los Lagos', 'Coquimbo'
]

print("Business parameters defined successfully.")

Business parameters defined successfully.


### 1.2 Generate Customer Data

In [3]:
# =============================================================================
# CUSTOMER DATA GENERATION
# =============================================================================

# Customer IDs
customer_ids = np.arange(1, N_CUSTOMERS + 1)

# Registration dates (days since 2023-01-01)
# Earlier customers more likely to be from MercadoLibre era
registration_days = np.random.exponential(scale=300, size=N_CUSTOMERS).astype(int)
registration_days = np.clip(registration_days, 0, 730)  # Max 2 years

# Customer regions (weighted towards major regions)
region_weights = np.array([0.40, 0.15, 0.12, 0.08, 0.07, 0.06, 0.07, 0.05])
customer_regions = np.random.choice(len(REGIONS), size=N_CUSTOMERS, p=region_weights)

# Acquisition channel (depends on registration date)
# Older customers more likely from MercadoLibre, newer from social media
def assign_channel(reg_day):
    """Assign acquisition channel based on registration date."""
    if reg_day < 180:  # First 6 months - MercadoLibre era
        weights = [0.20, 0.50, 0.15, 0.10, 0.05]
    elif reg_day < 365:  # Transition period
        weights = [0.15, 0.25, 0.25, 0.20, 0.15]
    else:  # Shopify era
        weights = [0.10, 0.10, 0.25, 0.30, 0.25]
    return np.random.choice(len(CHANNELS), p=weights)

# Vectorized channel assignment
customer_channels = np.array([assign_channel(d) for d in registration_days])

# Customer lifetime value score (0-100, calculated later based on transactions)
initial_cltv_score = np.zeros(N_CUSTOMERS)

print(f"Generated {N_CUSTOMERS} customers")
print(f"Registration days - Min: {registration_days.min()}, Max: {registration_days.max()}, Mean: {registration_days.mean():.1f}")

Generated 500 customers
Registration days - Min: 1, Max: 730, Mean: 276.5


### 1.3 Generate Transaction Data

In [4]:
# =============================================================================
# TRANSACTION DATA GENERATION
# =============================================================================

# Transaction IDs
transaction_ids = np.arange(1, N_TRANSACTIONS + 1)

# Assign customers to transactions (some customers buy more than others)
# Pareto principle: 20% of customers generate 80% of transactions
top_customers = customer_ids[:int(N_CUSTOMERS * 0.2)]
regular_customers = customer_ids[int(N_CUSTOMERS * 0.2):]

n_top_transactions = int(N_TRANSACTIONS * 0.6)
n_regular_transactions = N_TRANSACTIONS - n_top_transactions

transaction_customers = np.concatenate([
    np.random.choice(top_customers, size=n_top_transactions),
    np.random.choice(regular_customers, size=n_regular_transactions)
])
np.random.shuffle(transaction_customers)

# Transaction dates (days since 2023-01-01)
transaction_days = np.random.randint(0, 730, size=N_TRANSACTIONS)

# Platform assignment (earlier transactions more likely MercadoLibre)
platform_probabilities = np.where(transaction_days < 365, 0.7, 0.2)  # 70% ML before day 365
transaction_platforms = np.where(
    np.random.random(N_TRANSACTIONS) < platform_probabilities,
    0,  # mercadolibre
    1   # shopify
)

# Product selection
product_names = list(PRODUCTS.keys())
transaction_products = np.random.choice(len(product_names), size=N_TRANSACTIONS)

# Quantity per transaction (1-4 items)
transaction_quantities = np.random.choice([1, 1, 1, 2, 2, 3, 4], size=N_TRANSACTIONS)

# Unit prices based on product
def get_unit_price(product_idx):
    """Generate random price within product range."""
    product = product_names[product_idx]
    min_p = PRODUCTS[product]['min_price']
    max_p = PRODUCTS[product]['max_price']
    return np.random.randint(min_p, max_p + 1)

transaction_unit_prices = np.array([get_unit_price(p) for p in transaction_products])

# Calculate subtotals
transaction_subtotals = transaction_unit_prices * transaction_quantities

# Shipping costs
shipping_type = np.random.choice([0, 1], size=N_TRANSACTIONS, p=[0.7, 0.3])  # 70% standard
base_shipping = np.where(shipping_type == 0, SHIPPING_STANDARD, SHIPPING_EXPRESS)
transaction_shipping = np.where(transaction_subtotals >= FREE_SHIPPING_THRESHOLD, 0, base_shipping)

# Platform fees
transaction_fees = np.where(
    transaction_platforms == 0,
    transaction_subtotals * MERCADOLIBRE_FEE,
    transaction_subtotals * SHOPIFY_PAYMENT_FEE
)

# Total amount (customer pays)
transaction_totals = transaction_subtotals + transaction_shipping

# Net revenue (after fees)
transaction_net_revenue = transaction_subtotals - transaction_fees

print(f"Generated {N_TRANSACTIONS} transactions")
print(f"\nTransaction totals (CLP):")
print(f"  Min: ${transaction_totals.min():,.0f}")
print(f"  Max: ${transaction_totals.max():,.0f}")
print(f"  Mean: ${transaction_totals.mean():,.0f}")
print(f"  Total revenue: ${transaction_totals.sum():,.0f}")

Generated 2000 transactions

Transaction totals (CLP):
  Min: $6,501
  Max: $133,084
  Mean: $32,893
  Total revenue: $65,786,736


### 1.4 Basic NumPy Operations and Statistics

In [5]:
# =============================================================================
# BASIC NUMPY OPERATIONS
# =============================================================================

print("=" * 60)
print("BUSINESS METRICS USING NUMPY OPERATIONS")
print("=" * 60)

# Total metrics
print(f"\n1. AGGREGATE METRICS")
print(f"   Total transactions: {len(transaction_ids):,}")
print(f"   Total revenue: ${np.sum(transaction_totals):,.0f} CLP")
print(f"   Total fees paid: ${np.sum(transaction_fees):,.0f} CLP")
print(f"   Net revenue: ${np.sum(transaction_net_revenue):,.0f} CLP")

# Average metrics
print(f"\n2. AVERAGE METRICS")
print(f"   Average ticket: ${np.mean(transaction_totals):,.0f} CLP")
print(f"   Average items per transaction: {np.mean(transaction_quantities):.2f}")
print(f"   Average unit price: ${np.mean(transaction_unit_prices):,.0f} CLP")

# Platform comparison
print(f"\n3. PLATFORM COMPARISON")
ml_mask = transaction_platforms == 0
shopify_mask = transaction_platforms == 1

print(f"   MercadoLibre:")
print(f"     - Transactions: {np.sum(ml_mask):,}")
print(f"     - Revenue: ${np.sum(transaction_totals[ml_mask]):,.0f} CLP")
print(f"     - Avg ticket: ${np.mean(transaction_totals[ml_mask]):,.0f} CLP")

print(f"   Shopify:")
print(f"     - Transactions: {np.sum(shopify_mask):,}")
print(f"     - Revenue: ${np.sum(transaction_totals[shopify_mask]):,.0f} CLP")
print(f"     - Avg ticket: ${np.mean(transaction_totals[shopify_mask]):,.0f} CLP")

# Variability metrics
print(f"\n4. VARIABILITY METRICS")
print(f"   Std deviation (ticket): ${np.std(transaction_totals):,.0f} CLP")
print(f"   Variance (ticket): ${np.var(transaction_totals):,.0f} CLP²")
print(f"   Range: ${np.ptp(transaction_totals):,.0f} CLP")

BUSINESS METRICS USING NUMPY OPERATIONS

1. AGGREGATE METRICS
   Total transactions: 2,000
   Total revenue: $65,786,736 CLP
   Total fees paid: $4,499,933 CLP
   Net revenue: $56,207,303 CLP

2. AVERAGE METRICS
   Average ticket: $32,893 CLP
   Average items per transaction: 2.00
   Average unit price: $15,155 CLP

3. PLATFORM COMPARISON
   MercadoLibre:
     - Transactions: 892
     - Revenue: $29,104,160 CLP
     - Avg ticket: $32,628 CLP
   Shopify:
     - Transactions: 1,108
     - Revenue: $36,682,576 CLP
     - Avg ticket: $33,107 CLP

4. VARIABILITY METRICS
   Std deviation (ticket): $21,743 CLP
   Variance (ticket): $472,749,447 CLP²
   Range: $126,583 CLP


In [6]:
# =============================================================================
# PRODUCT ANALYSIS
# =============================================================================

print("\n5. PRODUCT ANALYSIS")
print("-" * 50)

for idx, product in enumerate(product_names):
    mask = transaction_products == idx
    count = np.sum(mask)
    revenue = np.sum(transaction_totals[mask])
    avg_price = np.mean(transaction_unit_prices[mask])
    
    print(f"   {product.upper()}:")
    print(f"     - Units sold: {count:,}")
    print(f"     - Revenue: ${revenue:,.0f} CLP")
    print(f"     - Avg unit price: ${avg_price:,.0f} CLP")
    print()


5. PRODUCT ANALYSIS
--------------------------------------------------
   SOCKS:
     - Units sold: 324
     - Revenue: $4,196,849 CLP
     - Avg unit price: $4,537 CLP

   TOWELS:
     - Units sold: 318
     - Revenue: $8,562,869 CLP
     - Avg unit price: $12,055 CLP

   TSHIRTS:
     - Units sold: 341
     - Revenue: $9,405,544 CLP
     - Avg unit price: $12,028 CLP

   SHORTS:
     - Units sold: 338
     - Revenue: $9,973,561 CLP
     - Avg unit price: $13,476 CLP

   JACKETS:
     - Units sold: 377
     - Revenue: $21,116,140 CLP
     - Avg unit price: $27,421 CLP

   PAJAMAS:
     - Units sold: 302
     - Revenue: $12,531,773 CLP
     - Avg unit price: $19,910 CLP



### 1.5 Save Generated Data

In [7]:
# =============================================================================
# SAVE DATA FOR NEXT LESSON
# =============================================================================

# Save as .npy files (NumPy native format)
data_path = '../data/raw/'

# Customer arrays
np.save(f'{data_path}customer_ids.npy', customer_ids)
np.save(f'{data_path}customer_registration_days.npy', registration_days)
np.save(f'{data_path}customer_regions.npy', customer_regions)
np.save(f'{data_path}customer_channels.npy', customer_channels)

# Transaction arrays
np.save(f'{data_path}transaction_ids.npy', transaction_ids)
np.save(f'{data_path}transaction_customers.npy', transaction_customers)
np.save(f'{data_path}transaction_days.npy', transaction_days)
np.save(f'{data_path}transaction_platforms.npy', transaction_platforms)
np.save(f'{data_path}transaction_products.npy', transaction_products)
np.save(f'{data_path}transaction_quantities.npy', transaction_quantities)
np.save(f'{data_path}transaction_unit_prices.npy', transaction_unit_prices)
np.save(f'{data_path}transaction_subtotals.npy', transaction_subtotals)
np.save(f'{data_path}transaction_shipping.npy', transaction_shipping)
np.save(f'{data_path}transaction_fees.npy', transaction_fees)
np.save(f'{data_path}transaction_totals.npy', transaction_totals)

# Save reference mappings as .npy with allow_pickle
np.save(f'{data_path}ref_regions.npy', np.array(REGIONS))
np.save(f'{data_path}ref_channels.npy', np.array(CHANNELS))
np.save(f'{data_path}ref_platforms.npy', np.array(PLATFORMS))
np.save(f'{data_path}ref_products.npy', np.array(product_names))

print("Data saved successfully to ../data/raw/")
print(f"\nFiles created:")
print(f"  - Customer arrays: 4 files")
print(f"  - Transaction arrays: 11 files")
print(f"  - Reference mappings: 4 files")

Data saved successfully to ../data/raw/

Files created:
  - Customer arrays: 4 files
  - Transaction arrays: 11 files
  - Reference mappings: 4 files


### 1.6 Lesson 1 Summary

**What we accomplished:**

1. ✅ Created synthetic customer data (500 customers) with realistic attributes
2. ✅ Generated transaction data (2,000 transactions) following business rules
3. ✅ Applied NumPy operations: sum, mean, std, variance, conditional selection
4. ✅ Saved arrays in .npy format for use in Lesson 2

**Why NumPy is efficient:**

- **Contiguous memory:** All elements stored together, faster access
- **Vectorization:** `transaction_subtotals = transaction_unit_prices * transaction_quantities` processes 2,000 multiplications in one line without loops
- **Broadcasting:** Automatic element-wise operations between arrays of different shapes
- **C-level speed:** Core operations 10-100x faster than Python loops

**Next step:** Lesson 2 - Convert these arrays to Pandas DataFrames for exploration and transformation.

---

## Lesson 2: Pandas - Data Exploration and Transformation

**Objective:** Load the NumPy arrays generated in Lesson 1, convert them to Pandas DataFrames, and perform initial exploration.

### Why Pandas?

While NumPy excels at numerical computations, Pandas provides:

1. **Labeled data:** Rows and columns have meaningful names
2. **Mixed data types:** Each column can have a different type
3. **Built-in data cleaning:** Methods for handling missing values, duplicates, etc.
4. **Powerful grouping:** Easy aggregation and pivot operations
5. **Time series support:** Native datetime handling

In [8]:
# Import Pandas
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print(f"Pandas version: {pd.__version__}")

Pandas version: 3.0.0


### 2.1 Load NumPy Arrays

In [9]:
# =============================================================================
# LOAD DATA FROM LESSON 1
# =============================================================================

data_path = '../data/raw/'

# Load reference mappings
REGIONS = np.load(f'{data_path}ref_regions.npy', allow_pickle=True)
CHANNELS = np.load(f'{data_path}ref_channels.npy', allow_pickle=True)
PLATFORMS = np.load(f'{data_path}ref_platforms.npy', allow_pickle=True)
PRODUCTS = np.load(f'{data_path}ref_products.npy', allow_pickle=True)

# Load customer arrays
customer_ids = np.load(f'{data_path}customer_ids.npy')
customer_registration_days = np.load(f'{data_path}customer_registration_days.npy')
customer_regions = np.load(f'{data_path}customer_regions.npy')
customer_channels = np.load(f'{data_path}customer_channels.npy')

# Load transaction arrays
transaction_ids = np.load(f'{data_path}transaction_ids.npy')
transaction_customers = np.load(f'{data_path}transaction_customers.npy')
transaction_days = np.load(f'{data_path}transaction_days.npy')
transaction_platforms = np.load(f'{data_path}transaction_platforms.npy')
transaction_products = np.load(f'{data_path}transaction_products.npy')
transaction_quantities = np.load(f'{data_path}transaction_quantities.npy')
transaction_unit_prices = np.load(f'{data_path}transaction_unit_prices.npy')
transaction_subtotals = np.load(f'{data_path}transaction_subtotals.npy')
transaction_shipping = np.load(f'{data_path}transaction_shipping.npy')
transaction_fees = np.load(f'{data_path}transaction_fees.npy')
transaction_totals = np.load(f'{data_path}transaction_totals.npy')

print("Data loaded successfully!")
print(f"  - Customers: {len(customer_ids)}")
print(f"  - Transactions: {len(transaction_ids)}")

Data loaded successfully!
  - Customers: 500
  - Transactions: 2000


### 2.2 Create DataFrames

In [10]:
# =============================================================================
# CREATE CUSTOMERS DATAFRAME
# =============================================================================

# Base date for converting days to actual dates
BASE_DATE = datetime(2023, 1, 1)

# Create customers DataFrame
df_customers = pd.DataFrame({
    'customer_id': customer_ids,
    'registration_date': [BASE_DATE + timedelta(days=int(d)) for d in customer_registration_days],
    'region': [REGIONS[i] for i in customer_regions],
    'acquisition_channel': [CHANNELS[i] for i in customer_channels]
})

print("Customers DataFrame created")
print(f"Shape: {df_customers.shape}")
df_customers.head(10)

Customers DataFrame created
Shape: (500, 4)


,customer_id,registration_date,region,acquisition_channel
0,1,2023-05-21,Araucania,organic
1,2,2024-12-31,Valparaiso,facebook_ads
2,3,2024-01-31,Metropolitana,instagram_ads
3,4,2023-10-01,Maule,facebook_ads
4,5,2023-02-20,Araucania,google_ads
5,6,2023-02-20,Metropolitana,mercadolibre_ads
6,7,2023-01-18,Los Lagos,mercadolibre_ads
7,8,2024-08-26,OHiggins,instagram_ads
8,9,2023-10-03,Los Lagos,mercadolibre_ads
9,10,2024-01-05,Araucania,facebook_ads


In [11]:
# =============================================================================
# CREATE TRANSACTIONS DATAFRAME
# =============================================================================

df_transactions = pd.DataFrame({
    'transaction_id': transaction_ids,
    'customer_id': transaction_customers,
    'transaction_date': [BASE_DATE + timedelta(days=int(d)) for d in transaction_days],
    'platform': [PLATFORMS[i] for i in transaction_platforms],
    'product': [PRODUCTS[i] for i in transaction_products],
    'quantity': transaction_quantities,
    'unit_price': transaction_unit_prices,
    'subtotal': transaction_subtotals,
    'shipping_cost': transaction_shipping,
    'platform_fee': transaction_fees,
    'total_amount': transaction_totals
})

print("Transactions DataFrame created")
print(f"Shape: {df_transactions.shape}")
df_transactions.head(10)

Transactions DataFrame created
Shape: (2000, 11)


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
0,1,364,2023-01-29,shopify,shorts,3,11870,35610,0,1068.30,35610
1,2,53,2023-04-22,mercadolibre,socks,1,3172,3172,5500,412.36,8672
2,3,19,2024-11-03,shopify,jackets,2,21834,43668,0,1310.04,43668
3,4,227,2023-05-11,shopify,jackets,2,31559,63118,0,1893.54,63118
4,5,24,2023-07-07,mercadolibre,socks,1,3061,3061,3500,397.93,6561
5,6,51,2024-11-05,shopify,socks,1,5756,5756,3500,172.68,9256
6,7,67,2024-12-09,shopify,tshirts,1,10666,10666,3500,319.98,14166
7,8,96,2024-04-19,shopify,jackets,3,29477,88431,0,2652.93,88431
8,9,333,2023-12-30,mercadolibre,shorts,3,16859,50577,0,6575.01,50577
9,10,10,2023-07-23,shopify,pajamas,2,22625,45250,0,1357.50,45250


### 2.3 Initial Exploration - First and Last Rows

In [12]:
# =============================================================================
# EXPLORE CUSTOMERS DATAFRAME
# =============================================================================
print("=" * 60)
print("CUSTOMERS DATAFRAME EXPLORATION")
print("=" * 60)

print("\n--- First 5 rows ---")
display(df_customers.head())

print("\n--- Last 5 rows ---")
display(df_customers.tail())

print("\n--- DataFrame Info ---")
df_customers.info()

CUSTOMERS DATAFRAME EXPLORATION

--- First 5 rows ---


,customer_id,registration_date,region,acquisition_channel
0,1,2023-05-21,Araucania,organic
1,2,2024-12-31,Valparaiso,facebook_ads
2,3,2024-01-31,Metropolitana,instagram_ads
3,4,2023-10-01,Maule,facebook_ads
4,5,2023-02-20,Araucania,google_ads



--- Last 5 rows ---


,customer_id,registration_date,region,acquisition_channel
495,496,2023-05-11,Metropolitana,mercadolibre_ads
496,497,2023-09-20,Los Lagos,google_ads
497,498,2023-01-25,Metropolitana,mercadolibre_ads
498,499,2024-12-31,Coquimbo,google_ads
499,500,2024-12-31,Valparaiso,instagram_ads



--- DataFrame Info ---
<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   customer_id          500 non-null    int64         
 1   registration_date    500 non-null    datetime64[us]
 2   region               500 non-null    str           
 3   acquisition_channel  500 non-null    str           
dtypes: datetime64[us](1), int64(1), str(2)
memory usage: 15.8 KB


In [13]:
# =============================================================================
# EXPLORE TRANSACTIONS DATAFRAME
# =============================================================================

print("=" * 60)
print("TRANSACTIONS DATAFRAME EXPLORATION")
print("=" * 60)

print("\n--- First 5 rows ---")
display(df_transactions.head())

print("\n--- Last 5 rows ---")
display(df_transactions.tail())

print("\n--- DataFrame Info ---")
df_transactions.info()

TRANSACTIONS DATAFRAME EXPLORATION

--- First 5 rows ---


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
0,1,364,2023-01-29,shopify,shorts,3,11870,35610,0,1068.30,35610
1,2,53,2023-04-22,mercadolibre,socks,1,3172,3172,5500,412.36,8672
2,3,19,2024-11-03,shopify,jackets,2,21834,43668,0,1310.04,43668
3,4,227,2023-05-11,shopify,jackets,2,31559,63118,0,1893.54,63118
4,5,24,2023-07-07,mercadolibre,socks,1,3061,3061,3500,397.93,6561



--- Last 5 rows ---


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
1995,1996,154,2024-07-12,shopify,jackets,3,31127,93381,0,2801.43,93381
1996,1997,74,2024-07-18,shopify,shorts,1,13559,13559,3500,406.77,17059
1997,1998,279,2023-07-06,shopify,socks,1,4256,4256,3500,127.68,7756
1998,1999,18,2024-07-03,shopify,towels,1,13301,13301,5500,399.03,18801
1999,2000,60,2024-03-28,shopify,jackets,2,29347,58694,0,1760.82,58694



--- DataFrame Info ---
<class 'pandas.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    2000 non-null   int64         
 1   customer_id       2000 non-null   int64         
 2   transaction_date  2000 non-null   datetime64[us]
 3   platform          2000 non-null   str           
 4   product           2000 non-null   str           
 5   quantity          2000 non-null   int64         
 6   unit_price        2000 non-null   int64         
 7   subtotal          2000 non-null   int64         
 8   shipping_cost     2000 non-null   int64         
 9   platform_fee      2000 non-null   float64       
 10  total_amount      2000 non-null   int64         
dtypes: datetime64[us](1), float64(1), int64(7), str(2)
memory usage: 172.0 KB


### 2.4 Descriptive Statistics

In [14]:
# =============================================================================
# DESCRIPTIVE STATISTICS - CUSTOMERS
# =============================================================================

print("=" * 60)
print("CUSTOMERS - DESCRIPTIVE STATISTICS")
print("=" * 60)

print("\n--- Numerical columns ---")
display(df_customers.describe())

print("\n--- Categorical columns ---")
print(f"\nRegion distribution:")
print(df_customers['region'].value_counts())

print(f"\nAcquisition channel distribution:")
print(df_customers['acquisition_channel'].value_counts())

CUSTOMERS - DESCRIPTIVE STATISTICS

--- Numerical columns ---


,customer_id,registration_date
count,500.000000,500
mean,250.500000,2023-10-04 12:31:40.800000
min,1.000000,2023-01-02 00:00:00
25%,125.750000,2023-03-24 18:00:00
50%,250.500000,2023-08-04 12:00:00
75%,375.250000,2024-02-27 18:00:00
max,500.000000,2024-12-31 00:00:00
std,144.481833,NaN



--- Categorical columns ---

Region distribution:
region
Metropolitana    214
Valparaiso        73
Biobio            67
Los Lagos         39
Araucania         34
Maule             30
OHiggins          27
Coquimbo          16
Name: count, dtype: int64

Acquisition channel distribution:
acquisition_channel
mercadolibre_ads    138
google_ads          104
facebook_ads        103
organic              81
instagram_ads        74
Name: count, dtype: int64


In [15]:
# =============================================================================
# DESCRIPTIVE STATISTICS - TRANSACTIONS
# =============================================================================

print("=" * 60)
print("TRANSACTIONS - DESCRIPTIVE STATISTICS")
print("=" * 60)

print("\n--- Numerical columns ---")
display(df_transactions.describe())

print("\n--- Platform distribution ---")
print(df_transactions['platform'].value_counts())

print("\n--- Product distribution ---")
print(df_transactions['product'].value_counts())

TRANSACTIONS - DESCRIPTIVE STATISTICS

--- Numerical columns ---


,transaction_id,customer_id,transaction_date,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
count,2000.000000,2000.000000,2000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1000.500000,148.052000,2023-12-30 01:25:40.800000,2.000500,15155.204000,30353.618000,2539.750000,2249.966540,32893.368000
min,1.000000,1.000000,2023-01-01 00:00:00,1.000000,2994.000000,3001.000000,0.000000,90.030000,6501.000000
25%,500.750000,39.000000,2023-06-29 00:00:00,1.000000,10329.500000,13025.250000,0.000000,623.527500,17188.000000
50%,1000.500000,84.000000,2024-01-01 12:00:00,2.000000,13773.000000,23640.500000,3500.000000,1357.415000,27450.500000
75%,1500.250000,247.000000,2024-06-25 00:00:00,3.000000,20437.500000,42114.500000,3500.000000,2793.285000,42114.500000
max,2000.000000,500.000000,2024-12-30 00:00:00,4.000000,34809.000000,133084.000000,5500.000000,17087.720000,133084.000000
std,577.494589,143.913209,NaN,1.053591,7771.897543,23286.194654,2113.061157,2579.073174,21748.239935



--- Platform distribution ---
platform
shopify         1108
mercadolibre     892
Name: count, dtype: int64

--- Product distribution ---
product
jackets    377
tshirts    341
shorts     338
socks      324
towels     318
pajamas    302
Name: count, dtype: int64


### 2.5 Conditional Filtering

In [16]:
# =============================================================================
# CONDITIONAL FILTERS - BUSINESS QUESTIONS
# =============================================================================

print("=" * 60)
print("BUSINESS QUESTIONS WITH CONDITIONAL FILTERS")
print("=" * 60)

# Q1: High-value transactions (above 50,000 CLP)
high_value = df_transactions[df_transactions['total_amount'] > 50000]
print(f"\n1. High-value transactions (>$50,000 CLP): {len(high_value)}")
print(f"   Revenue from high-value: ${high_value['total_amount'].sum():,.0f} CLP")

# Q2: MercadoLibre transactions in 2023
ml_2023 = df_transactions[
    (df_transactions['platform'] == 'mercadolibre') & 
    (df_transactions['transaction_date'].dt.year == 2023)
]
print(f"\n2. MercadoLibre transactions in 2023: {len(ml_2023)}")

# Q3: Shopify transactions in 2024
shopify_2024 = df_transactions[
    (df_transactions['platform'] == 'shopify') & 
    (df_transactions['transaction_date'].dt.year == 2024)
]
print(f"\n3. Shopify transactions in 2024: {len(shopify_2024)}")

# Q4: Customers from Metropolitana region acquired via Instagram
metro_ig = df_customers[
    (df_customers['region'] == 'Metropolitana') & 
    (df_customers['acquisition_channel'] == 'instagram_ads')
]
print(f"\n4. Metropolitana customers from Instagram: {len(metro_ig)}")

# Q5: Jacket sales with quantity > 1
jacket_multi = df_transactions[
    (df_transactions['product'] == 'jackets') & 
    (df_transactions['quantity'] > 1)
]
print(f"\n5. Jacket sales with multiple units: {len(jacket_multi)}")
print(f"   Total jackets sold in bulk: {jacket_multi['quantity'].sum()}")

BUSINESS QUESTIONS WITH CONDITIONAL FILTERS

1. High-value transactions (>$50,000 CLP): 358
   Revenue from high-value: $25,381,355 CLP

2. MercadoLibre transactions in 2023: 676

3. Shopify transactions in 2024: 788

4. Metropolitana customers from Instagram: 35

5. Jacket sales with multiple units: 232
   Total jackets sold in bulk: 613


In [17]:
# =============================================================================
# ADVANCED FILTERS - USING QUERY METHOD
# =============================================================================

print("\n--- Using query() method for cleaner syntax ---")

# Transactions with free shipping (subtotal >= 30,000)
free_shipping = df_transactions.query('shipping_cost == 0')
print(f"\n6. Transactions with free shipping: {len(free_shipping)}")
print(f"   Percentage: {len(free_shipping)/len(df_transactions)*100:.1f}%")

# Low-value socks transactions
low_socks = df_transactions.query('product == "socks" and total_amount < 10000')
print(f"\n7. Low-value socks transactions (<$10,000): {len(low_socks)}")


--- Using query() method for cleaner syntax ---

6. Transactions with free shipping: 759
   Percentage: 38.0%

7. Low-value socks transactions (<$10,000): 123


### 2.6 Save Preliminary DataFrames

In [18]:
# =============================================================================
# SAVE DATAFRAMES AS CSV
# =============================================================================

# Save to raw folder (these are still "raw" DataFrames, not yet cleaned)
df_customers.to_csv('../data/raw/customers_preliminary.csv', index=False)
df_transactions.to_csv('../data/raw/transactions_preliminary.csv', index=False)

print("DataFrames saved successfully!")
print(f"  - customers_preliminary.csv ({len(df_customers)} rows)")
print(f"  - transactions_preliminary.csv ({len(df_transactions)} rows)")

DataFrames saved successfully!
  - customers_preliminary.csv (500 rows)
  - transactions_preliminary.csv (2000 rows)


### 2.7 Lesson 2 Summary

**What we accomplished:**

1. ✅ Loaded NumPy arrays from Lesson 1
2. ✅ Converted arrays to Pandas DataFrames with proper column names
3. ✅ Transformed day numbers to actual datetime objects
4. ✅ Mapped numeric indices to categorical labels (regions, channels, platforms, products)
5. ✅ Explored data with head(), tail(), info(), describe()
6. ✅ Applied conditional filters to answer business questions
7. ✅ Saved preliminary CSVs for next lesson

**Key Pandas methods used:**

- `pd.DataFrame()` - Create DataFrame from dictionary
- `.head()`, `.tail()` - View first/last rows
- `.info()` - DataFrame structure and memory usage
- `.describe()` - Statistical summary
- `.value_counts()` - Frequency distribution
- Boolean indexing `df_treated[condition]` - Filter rows
- `.query()` - SQL-like filtering syntax
- `.to_csv()` - Export to CSV file

**Key findings:**

- Customer distribution is weighted towards Metropolitana region (as expected)
- Platform migration from MercadoLibre to Shopify is visible in the data
- Free shipping threshold impacts a significant portion of transactions

**Next step:** Lesson 3 - Integrate data from additional sources (Excel and web tables).

---

## Lesson 3: Data Acquisition from Files

**Objective:** Integrate data from diverse sources (CSV, Excel, web tables) and unify them into a single DataFrame for subsequent cleaning.

### Data Sources Overview

| Source | File | Format | Description |
|--------|------|--------|-------------|
| MercadoLibre | transactions_preliminary.csv | CSV | Historical transactions from Lesson 2 |
| Shopify | shopify_orders_2024.xlsx | Excel | Orders from the Shopify platform |
| Marketing | marketing_metrics.html | Web Table | Campaign performance data |

In [19]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

print("Libraries loaded for Lesson 3")

Libraries loaded for Lesson 3


### 3.1 Load CSV File (MercadoLibre Data)

In [20]:
# =============================================================================
# LOAD CSV - MERCADOLIBRE TRANSACTIONS
# =============================================================================

df_mercadolibre = pd.read_csv('../data/raw/transactions_preliminary.csv')

# Filter only MercadoLibre transactions
df_mercadolibre = df_mercadolibre[df_mercadolibre['platform'] == 'mercadolibre'].copy()

print("MercadoLibre CSV loaded")
print(f"Shape: {df_mercadolibre.shape}")
print(f"\nColumns: {list(df_mercadolibre.columns)}")
df_mercadolibre.head()

MercadoLibre CSV loaded
Shape: (892, 11)

Columns: ['transaction_id', 'customer_id', 'transaction_date', 'platform', 'product', 'quantity', 'unit_price', 'subtotal', 'shipping_cost', 'platform_fee', 'total_amount']


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
1,2,53,2023-04-22,mercadolibre,socks,1,3172,3172,5500,412.36,8672
4,5,24,2023-07-07,mercadolibre,socks,1,3061,3061,3500,397.93,6561
8,9,333,2023-12-30,mercadolibre,shorts,3,16859,50577,0,6575.01,50577
10,11,5,2023-07-31,mercadolibre,towels,2,14402,28804,5500,3744.52,34304
13,14,44,2023-03-14,mercadolibre,jackets,1,23032,23032,3500,2994.16,26532


### 3.2 Load Excel File (Shopify Data)

In [21]:
# =============================================================================
# LOAD EXCEL - SHOPIFY ORDERS
# =============================================================================

df_shopify = pd.read_excel('../data/raw/shopify_orders_2024.xlsx', sheet_name='Orders')

print("Shopify Excel loaded")
print(f"Shape: {df_shopify.shape}")
print(f"\nColumns: {list(df_shopify.columns)}")
df_shopify.head()

Shopify Excel loaded
Shape: (300, 14)

Columns: ['Order Number', 'Order Date', 'Customer Email', 'Customer Name', 'Product Title', 'Quantity', 'Unit Price (CLP)', 'Discount', 'Shipping', 'Payment Method', 'Fulfillment Status', 'Region', 'Subtotal', 'Total (CLP)']


,Order Number,Order Date,Customer Email,Customer Name,Product Title,Quantity,Unit Price (CLP),Discount,Shipping,Payment Method,Fulfillment Status,Region,Subtotal,Total (CLP)
0,PS-10000,31/12/2024,cliente370@email.com,Cliente 365,Chaqueta Polar,1,4990,500,0,Webpay,Fulfilled,RM,4990,4490
1,PS-10001,17/01/2025,cliente84@email.com,Cliente 154,Calcetines Niño,1,13990,0,3500,Webpay,Fulfilled,X,13990,17490
2,PS-10002,18/11/2024,cliente351@email.com,Cliente 33,Short Verano,1,27990,1500,5500,MercadoPago,Fulfilled,VIII,27990,31990
3,PS-10003,08/04/2024,cliente72@email.com,Cliente 188,Short Verano,1,13990,0,3500,Webpay,Pending,IX,13990,17490
4,PS-10004,18/08/2024,cliente40@email.com,Cliente 393,Chaqueta Polar,1,27990,1000,3500,MercadoPago,Fulfilled,RM,27990,30490


In [22]:
# Check data types and info
print("Shopify DataFrame Info:")
df_shopify.info()

Shopify DataFrame Info:
<class 'pandas.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Order Number        300 non-null    str  
 1   Order Date          300 non-null    str  
 2   Customer Email      297 non-null    str  
 3   Customer Name       299 non-null    str  
 4   Product Title       300 non-null    str  
 5   Quantity            300 non-null    int64
 6   Unit Price (CLP)    300 non-null    int64
 7   Discount            300 non-null    int64
 8   Shipping            300 non-null    int64
 9   Payment Method      300 non-null    str  
 10  Fulfillment Status  300 non-null    str  
 11  Region              298 non-null    str  
 12  Subtotal            300 non-null    int64
 13  Total (CLP)         300 non-null    int64
dtypes: int64(6), str(8)
memory usage: 32.9 KB


### 3.3 Load Web Table (Marketing Data)

In [23]:
# =============================================================================
# LOAD HTML TABLE - MARKETING METRICS
# =============================================================================

# read_html returns a list of DataFrames (one per table found)
tables = pd.read_html('../data/raw/marketing_metrics.html')

print(f"Found {len(tables)} table(s) in HTML file")

# Get the first (and only) table
df_marketing = tables[0]

print(f"\nMarketing table loaded")
print(f"Shape: {df_marketing.shape}")
print(f"\nColumns: {list(df_marketing.columns)}")
df_marketing.head(10)

Found 1 table(s) in HTML file

Marketing table loaded
Shape: (24, 10)

Columns: ['Campaign', 'Channel', 'Month', 'Impressions', 'Clicks', 'CTR (%)', 'Spend (CLP)', 'Conversions', 'Revenue (CLP)', 'ROAS']


,Campaign,Channel,Month,Impressions,Clicks,CTR (%),Spend (CLP),Conversions,Revenue (CLP),ROAS
0,Summer Collection,Facebook,Jan-24,125000,3750,3.0,450000,45,1350000,3.0
1,Summer Collection,Instagram,Jan-24,98000,4410,4.5,380000,52,1560000,4.1
2,Summer Collection,Google,Jan-24,85000,2550,3.0,320000,38,1140000,3.6
3,Back to School,Facebook,Feb-24,145000,5075,3.5,520000,62,1860000,3.6
4,Back to School,Instagram,Feb-24,112000,5600,5.0,420000,71,2130000,5.1
5,Back to School,Google,Feb-24,95000,3325,3.5,380000,48,1440000,3.8
6,Fall Essentials,Facebook,Apr-24,88000,2640,3.0,280000,33,990000,3.5
7,Fall Essentials,Instagram,Apr-24,76000,3420,4.5,250000,41,1230000,4.9
8,Fall Essentials,Google,Apr-24,62000,1860,3.0,200000,25,750000,3.8
9,Winter Warmth,Facebook,Jun-24,135000,4725,3.5,480000,58,2030000,4.2


In [24]:
# Explore marketing data
print("Marketing DataFrame Info:")
df_marketing.info()

print("\n--- Campaign distribution ---")
print(df_marketing['Campaign'].value_counts())

print("\n--- Channel distribution ---")
print(df_marketing['Channel'].value_counts())

Marketing DataFrame Info:
<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Campaign       24 non-null     str    
 1   Channel        24 non-null     str    
 2   Month          24 non-null     str    
 3   Impressions    24 non-null     int64  
 4   Clicks         24 non-null     int64  
 5   CTR (%)        24 non-null     float64
 6   Spend (CLP)    24 non-null     int64  
 7   Conversions    24 non-null     int64  
 8   Revenue (CLP)  24 non-null     int64  
 9   ROAS           24 non-null     float64
dtypes: float64(2), int64(5), str(3)
memory usage: 2.0 KB

--- Campaign distribution ---
Campaign
Winter Warmth        4
Cyber Monday         4
Christmas Magic      4
Summer Collection    3
Back to School       3
Fall Essentials      3
Spring Preview       3
Name: count, dtype: int64

--- Channel distribution ---
Channel
Facebook        7
Instagram    

### 3.4 Data Harmonization Challenges

Before unifying the data, let's identify the challenges:

In [25]:
# =============================================================================
# IDENTIFY DATA HARMONIZATION CHALLENGES
# =============================================================================

print("=" * 60)
print("DATA HARMONIZATION CHALLENGES")
print("=" * 60)

print("\n1. COLUMN NAME DIFFERENCES")
print(f"   MercadoLibre: {list(df_mercadolibre.columns)[:5]}...")
print(f"   Shopify: {list(df_shopify.columns)[:5]}...")

print("\n2. DATE FORMAT DIFFERENCES")
print(f"   MercadoLibre date sample: {df_mercadolibre['transaction_date'].iloc[0]}")
print(f"   Shopify date sample: {df_shopify['Order Date'].iloc[0]}")

print("\n3. PRODUCT NAME DIFFERENCES")
print(f"   MercadoLibre products: {df_mercadolibre['product'].unique()[:3]}...")
print(f"   Shopify products: {df_shopify['Product Title'].unique()[:3]}...")

print("\n4. REGION FORMAT DIFFERENCES")
print(f"   Shopify regions: {df_shopify['Region'].unique()}")

print("\n5. NULL VALUES IN SHOPIFY DATA")
print(df_shopify.isnull().sum()[df_shopify.isnull().sum() > 0])

DATA HARMONIZATION CHALLENGES

1. COLUMN NAME DIFFERENCES
   MercadoLibre: ['transaction_id', 'customer_id', 'transaction_date', 'platform', 'product']...
   Shopify: ['Order Number', 'Order Date', 'Customer Email', 'Customer Name', 'Product Title']...

2. DATE FORMAT DIFFERENCES
   MercadoLibre date sample: 2023-04-22
   Shopify date sample: 31/12/2024

3. PRODUCT NAME DIFFERENCES
   MercadoLibre products: <StringArray>
['socks', 'shorts', 'towels']
Length: 3, dtype: str...
   Shopify products: <StringArray>
['Chaqueta Polar', 'Calcetines Niño', 'Short Verano']
Length: 3, dtype: str...

4. REGION FORMAT DIFFERENCES
   Shopify regions: <StringArray>
['RM', 'X', 'VIII', 'IX', 'Metr', 'IV', 'V', 'VI', 'VII', 'metr', nan, 'Valp']
Length: 12, dtype: str

5. NULL VALUES IN SHOPIFY DATA
Customer Email    3
Customer Name     1
Region            2
dtype: int64


### 3.5 Standardize Shopify Data

In [26]:
# =============================================================================
# STANDARDIZE SHOPIFY DATA TO MATCH MERCADOLIBRE FORMAT
# =============================================================================

# Create a copy for transformation
df_shopify_std = df_shopify.copy()

# 1. Rename columns to match MercadoLibre format
column_mapping = {
    'Order Number': 'transaction_id',
    'Order Date': 'transaction_date',
    'Product Title': 'product',
    'Quantity': 'quantity',
    'Unit Price (CLP)': 'unit_price',
    'Subtotal': 'subtotal',
    'Shipping': 'shipping_cost',
    'Total (CLP)': 'total_amount',
    'Region': 'region'
}
df_shopify_std = df_shopify_std.rename(columns=column_mapping)

# 2. Add platform column
df_shopify_std['platform'] = 'shopify'

# 3. Convert date format (from DD/MM/YYYY to datetime)
df_shopify_std['transaction_date'] = pd.to_datetime(
    df_shopify_std['transaction_date'], 
    format='%d/%m/%Y',
    errors='coerce'
)

# 4. Standardize product names (Spanish to English)
product_mapping = {
    'Calcetines Niño': 'socks',
    'Toalla Playa Disney': 'towels',
    'Polera Estampada': 'tshirts',
    'Short Verano': 'shorts',
    'Chaqueta Polar': 'jackets',
    'Pijama Algodón': 'pajamas'
}
df_shopify_std['product'] = df_shopify_std['product'].map(product_mapping)

# 5. Standardize region names
region_mapping = {
    'RM': 'Metropolitana',
    'Metropolitana': 'Metropolitana',
    'metropolitana': 'Metropolitana',
    'V': 'Valparaiso',
    'Valparaíso': 'Valparaiso',
    'VIII': 'Biobio',
    'IX': 'Araucania',
    'VII': 'Maule',
    'VI': 'OHiggins',
    'X': 'Los Lagos',
    'IV': 'Coquimbo'
}
df_shopify_std['region'] = df_shopify_std['region'].map(region_mapping)

# 6. Calculate platform fee (3% for Shopify payment gateway)
df_shopify_std['platform_fee'] = df_shopify_std['subtotal'] * 0.03

# 6.5 Generate customer_id for Shopify (continuing from MercadoLibre max)
np.random.seed(789)
existing_customers = df_customers['customer_id'].values
df_shopify_std['customer_id'] = np.random.choice(existing_customers, size=len(df_shopify_std))

# 7. Select and reorder columns to match MercadoLibre
columns_to_keep = [
    'transaction_id',  'customer_id', 'transaction_date', 'platform', 'product',
    'quantity', 'unit_price', 'subtotal', 'shipping_cost', 
    'platform_fee', 'total_amount'
]
df_shopify_std = df_shopify_std[columns_to_keep]

print("Shopify data standardized")
print(f"Shape: {df_shopify_std.shape}")
df_shopify_std.head()

Shopify data standardized
Shape: (300, 11)


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
0,PS-10000,52,2024-12-31,shopify,jackets,1,4990,4990,0,149.7,4490
1,PS-10001,143,2025-01-17,shopify,socks,1,13990,13990,3500,419.7,17490
2,PS-10002,415,2024-11-18,shopify,shorts,1,27990,27990,5500,839.7,31990
3,PS-10003,211,2024-04-08,shopify,shorts,1,13990,13990,3500,419.7,17490
4,PS-10004,386,2024-08-18,shopify,jackets,1,27990,27990,3500,839.7,30490


### 3.6 Standardize MercadoLibre Data

In [27]:
# =============================================================================
# STANDARDIZE MERCADOLIBRE DATA
# =============================================================================

df_ml_std = df_mercadolibre.copy()

# Convert date to datetime if not already
df_ml_std['transaction_date'] = pd.to_datetime(df_ml_std['transaction_date'])

# Select matching columns
columns_to_keep = [
    'transaction_id', 'customer_id', 'transaction_date', 'platform', 'product',
    'quantity', 'unit_price', 'subtotal', 'shipping_cost', 
    'platform_fee', 'total_amount'
]
df_ml_std = df_ml_std[columns_to_keep]

print("MercadoLibre data standardized")
print(f"Shape: {df_ml_std.shape}")
df_ml_std.head()

MercadoLibre data standardized
Shape: (892, 11)


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
1,2,53,2023-04-22,mercadolibre,socks,1,3172,3172,5500,412.36,8672
4,5,24,2023-07-07,mercadolibre,socks,1,3061,3061,3500,397.93,6561
8,9,333,2023-12-30,mercadolibre,shorts,3,16859,50577,0,6575.01,50577
10,11,5,2023-07-31,mercadolibre,towels,2,14402,28804,5500,3744.52,34304
13,14,44,2023-03-14,mercadolibre,jackets,1,23032,23032,3500,2994.16,26532


### 3.7 Unify Transaction Data with concat()

In [28]:
# =============================================================================
# UNIFY DATA SOURCES USING CONCAT
# =============================================================================


# Concatenate MercadoLibre and Shopify transactions
df_transactions_unified = pd.concat(
    [df_ml_std, df_shopify_std], 
    ignore_index=True
)

# Sort by date
df_transactions_unified = df_transactions_unified.sort_values('transaction_date').reset_index(drop=True)

print("=" * 60)
print("UNIFIED TRANSACTIONS DATAFRAME")
print("=" * 60)
print(f"\nTotal rows: {len(df_transactions_unified)}")
print(f"  - From MercadoLibre: {len(df_ml_std)}")
print(f"  - From Shopify: {len(df_shopify_std)}")

print(f"\nDate range: {df_transactions_unified['transaction_date'].min()} to {df_transactions_unified['transaction_date'].max()}")

print("\n--- Platform distribution ---")
print(df_transactions_unified['platform'].value_counts())

df_transactions_unified.head(10)

UNIFIED TRANSACTIONS DATAFRAME

Total rows: 1192
  - From MercadoLibre: 892
  - From Shopify: 300

Date range: 2023-01-01 00:00:00 to 2025-01-30 00:00:00

--- Platform distribution ---
platform
mercadolibre    892
shopify         300
Name: count, dtype: int64


,transaction_id,customer_id,transaction_date,platform,product,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
0,681,30,2023-01-01,mercadolibre,pajamas,3,22008,66024,0,8583.12,66024
1,995,54,2023-01-01,mercadolibre,jackets,2,28273,56546,0,7350.98,56546
2,1800,57,2023-01-01,mercadolibre,socks,4,3904,15616,5500,2030.08,21116
3,308,199,2023-01-01,mercadolibre,jackets,1,30474,30474,0,3961.62,30474
4,700,18,2023-01-03,mercadolibre,shorts,1,13211,13211,3500,1717.43,16711
5,1490,80,2023-01-04,mercadolibre,socks,1,5961,5961,3500,774.93,9461
6,1852,75,2023-01-04,mercadolibre,shorts,4,13457,53828,0,6997.64,53828
7,1378,77,2023-01-05,mercadolibre,tshirts,2,11994,23988,3500,3118.44,27488
8,82,421,2023-01-07,mercadolibre,pajamas,1,18720,18720,3500,2433.60,22220
9,1590,87,2023-01-07,mercadolibre,tshirts,1,11711,11711,3500,1522.43,15211


In [29]:
# Verify data integrity
print("\n--- Data Types ---")
print(df_transactions_unified.dtypes)

print("\n--- Null Values ---")
print(df_transactions_unified.isnull().sum())

print("\n--- Descriptive Statistics ---")
df_transactions_unified.describe()


--- Data Types ---
transaction_id              object
customer_id                  int64
transaction_date    datetime64[us]
platform                       str
product                        str
quantity                     int64
unit_price                   int64
subtotal                     int64
shipping_cost                int64
platform_fee               float64
total_amount                 int64
dtype: object

--- Null Values ---
transaction_id      0
customer_id         0
transaction_date    0
platform            0
product             0
quantity            0
unit_price          0
subtotal            0
shipping_cost       0
platform_fee        0
total_amount        0
dtype: int64

--- Descriptive Statistics ---


,customer_id,transaction_date,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
count,1192.000000,1192,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000
mean,170.283557,2023-12-10 08:26:10.469798,1.952181,15364.786074,29108.775168,2746.644295,3120.508221,31688.892617
min,1.000000,2023-01-01 00:00:00,1.000000,3010.000000,3039.000000,0.000000,149.700000,4490.000000
25%,44.750000,2023-06-06 18:00:00,1.000000,10600.000000,12990.000000,0.000000,933.595000,16564.750000
50%,98.000000,2023-11-16 12:00:00,2.000000,13712.500000,22784.500000,3500.000000,1973.400000,26333.500000
75%,294.000000,2024-06-06 06:00:00,3.000000,19990.000000,39033.500000,3500.000000,4148.950000,39525.000000
max,500.000000,2025-01-30 00:00:00,50.000000,299900.000000,299900.000000,5500.000000,17087.720000,299900.000000
std,151.641979,NaN,1.719224,11220.781024,24177.432748,2063.941469,2999.539522,23050.424057


### 3.8 Save Consolidated Data

In [30]:
# =============================================================================
# SAVE CONSOLIDATED DATAFRAMES
# =============================================================================

# Save unified transactions
df_transactions_unified.to_csv('../data/raw/transactions_consolidated.csv', index=False)

# Save marketing data
df_marketing.to_csv('../data/raw/marketing_metrics.csv', index=False)

print("Consolidated data saved:")
print(f"  - transactions_consolidated.csv ({len(df_transactions_unified)} rows)")
print(f"  - marketing_metrics.csv ({len(df_marketing)} rows)")

# Verify that customer_id is in the saved file
df_test = pd.read_csv('../data/raw/transactions_consolidated.csv')
print(f"Columns: {list(df_test.columns)}")
print(f"customer_id present: {'customer_id' in df_test.columns}")

Consolidated data saved:
  - transactions_consolidated.csv (1192 rows)
  - marketing_metrics.csv (24 rows)
Columns: ['transaction_id', 'customer_id', 'transaction_date', 'platform', 'product', 'quantity', 'unit_price', 'subtotal', 'shipping_cost', 'platform_fee', 'total_amount']
customer_id present: True


### 3.9 Lesson 3 Summary

**What we accomplished:**

1. ✅ Loaded CSV file with `pd.read_csv()` - MercadoLibre transactions
2. ✅ Loaded Excel file with `pd.read_excel()` - Shopify orders
3. ✅ Loaded HTML table with `pd.read_html()` - Marketing metrics
4. ✅ Identified harmonization challenges (column names, date formats, product names, regions)
5. ✅ Standardized column names and data formats
6. ✅ Unified transaction data using `pd.concat()`
7. ✅ Saved consolidated datasets

**Challenges encountered:**

- Different column naming conventions between platforms
- Date format differences (DD/MM/YYYY vs YYYY-MM-DD)
- Product names in Spanish (Shopify) vs English (MercadoLibre)
- Region codes vs full names
- Null values in Shopify data (to be handled in Lesson 4)

**Key Pandas methods used:**

- `pd.read_csv()` - Load CSV files
- `pd.read_excel()` - Load Excel files
- `pd.read_html()` - Parse HTML tables
- `.rename()` - Rename columns
- `.map()` - Map values using dictionary
- `pd.to_datetime()` - Convert to datetime
- `pd.concat()` - Combine DataFrames vertically

**Next step:** Lesson 4 - Handle missing values and outliers in the consolidated data.

---

## Lesson 4: Missing Values and Outliers

**Objective:** Apply data cleaning techniques to resolve null values and detect/treat outliers in the consolidated dataset.

### Techniques to Apply

**For Missing Values:**
- Identification with `.isnull()` and `.isna()`
- Elimination with `.dropna()`
- Imputation with `.fillna()` (mean, median, mode, forward fill)

**For Outliers:**
- IQR (Interquartile Range) method
- Z-score method

In [31]:
# Import libraries
import pandas as pd
import numpy as np
from scipy import stats

print("Libraries loaded for Lesson 4")

Libraries loaded for Lesson 4


### 4.1 Load Consolidated Data

In [32]:
# =============================================================================
# LOAD CONSOLIDATED DATA FROM LESSON 3
# =============================================================================
df = pd.read_csv('../data/raw/transactions_consolidated.csv')

# Convert date column
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

print(f"Loaded {len(df)} transactions")
print(f"\nColumns: {list(df.columns)}")
df.head()
print('customer_id' in df.columns)

Loaded 1192 transactions

Columns: ['transaction_id', 'customer_id', 'transaction_date', 'platform', 'product', 'quantity', 'unit_price', 'subtotal', 'shipping_cost', 'platform_fee', 'total_amount']
True


### 4.2 Identify Missing Values

In [33]:
# =============================================================================
# IDENTIFY NULL VALUES
# =============================================================================
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Count nulls per column
null_counts = df.isnull().sum()
null_percentages = (df.isnull().sum() / len(df) * 100).round(2)

# Create summary DataFrame
null_summary = pd.DataFrame({
    'Null Count': null_counts,
    'Null %': null_percentages
})

print("\n--- Null Values Summary ---")
print(null_summary)

# Total rows with at least one null
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"\nRows with at least one null value: {rows_with_nulls} ({rows_with_nulls/len(df)*100:.2f}%)")

MISSING VALUES ANALYSIS

--- Null Values Summary ---
                  Null Count  Null %
transaction_id             0     0.0
customer_id                0     0.0
transaction_date           0     0.0
platform                   0     0.0
product                    0     0.0
quantity                   0     0.0
unit_price                 0     0.0
subtotal                   0     0.0
shipping_cost              0     0.0
platform_fee               0     0.0
total_amount               0     0.0

Rows with at least one null value: 0 (0.00%)


In [34]:
# Visualize rows with null values
print("\n--- Sample Rows with Null Values ---")
null_rows = df[df.isnull().any(axis=1)]
if len(null_rows) > 0:
    display(null_rows.head(10))
else:
    print("No null values found in the dataset.")


--- Sample Rows with Null Values ---
No null values found in the dataset.


### 4.3 Handle Missing Values

In [35]:
# =============================================================================
# HANDLE MISSING VALUES - STRATEGY BY COLUMN
# =============================================================================

# Create a working copy
df_clean =  df.copy()

print("=" * 60)
print("MISSING VALUES TREATMENT")
print("=" * 60)

# Strategy for each column type:
# - Numerical: impute with median (robust to outliers)
# - Categorical: impute with mode or 'Unknown'
# - Critical fields: drop rows if null

# Check which columns have nulls
columns_with_nulls = df_clean.columns[df_clean.isnull().any()].tolist()

if len(columns_with_nulls) > 0:
    print(f"\nColumns with null values: {columns_with_nulls}")
    
    for col in columns_with_nulls:
        null_count = df_clean[col].isnull().sum()
        
        if df_clean[col].dtype in ['int64', 'float64']:
            # Numerical: impute with median
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(f"  - {col}: Imputed {null_count} nulls with median ({median_val:.2f})")
        else:
            # Categorical: impute with mode
            mode_val = df_clean[col].mode()[0] if len(df_clean[col].mode()) > 0 else 'Unknown'
            df_clean[col] = df_clean[col].fillna(mode_val)
            print(f"  - {col}: Imputed {null_count} nulls with mode ('{mode_val}')")
else:
    print("\nNo null values to handle.")

# Verify no nulls remain
print(f"\nRemaining null values: {df_clean.isnull().sum().sum()}")

MISSING VALUES TREATMENT

No null values to handle.

Remaining null values: 0


### 4.4 Detect Outliers - IQR Method

In [36]:
# =============================================================================
# OUTLIER DETECTION - IQR METHOD
# =============================================================================

def detect_outliers_iqr(data, column):
    """
    Detect outliers using the IQR method.
    
    Outliers are values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR.
    
    Parameters:
        data: DataFrame
        column: Column name to analyze
    
    Returns:
        Dictionary with outlier statistics
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_mask = (data[column] < lower_bound) | (data[column] > upper_bound)
    outliers = data[outliers_mask]
    
    return {
        'column': column,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'n_outliers': len(outliers),
        'outlier_indices': outliers.index.tolist(),
        'outlier_values': outliers[column].tolist()
    }

print("=" * 60)
print("OUTLIER DETECTION - IQR METHOD")
print("=" * 60)

# Numerical columns to check for outliers
numerical_cols = ['quantity', 'unit_price', 'subtotal', 'shipping_cost', 'platform_fee', 'total_amount']

outlier_results = []

for col in numerical_cols:
    result = detect_outliers_iqr(df_clean, col)
    outlier_results.append(result)
    
    if result['n_outliers'] > 0:
        print(f"\n{col.upper()}:")
        print(f"  Q1: {result['Q1']:,.2f}, Q3: {result['Q3']:,.2f}, IQR: {result['IQR']:,.2f}")
        print(f"  Bounds: [{result['lower_bound']:,.2f}, {result['upper_bound']:,.2f}]")
        print(f"  Outliers found: {result['n_outliers']}")
        if result['n_outliers'] <= 10:
            print(f"  Values: {result['outlier_values']}")

OUTLIER DETECTION - IQR METHOD

QUANTITY:
  Q1: 1.00, Q3: 3.00, IQR: 2.00
  Bounds: [-2.00, 6.00]
  Outliers found: 1
  Values: [50]

UNIT_PRICE:
  Q1: 10,600.00, Q3: 19,990.00, IQR: 9,390.00
  Bounds: [-3,485.00, 34,075.00]
  Outliers found: 10
  Values: [34235, 34260, 34659, 34209, 34755, 34809, 299900, 34766, 34146, 34274]

SUBTOTAL:
  Q1: 12,990.00, Q3: 39,033.50, IQR: 26,043.50
  Bounds: [-26,075.25, 78,098.75]
  Outliers found: 52

PLATFORM_FEE:
  Q1: 933.60, Q3: 4,148.95, IQR: 3,215.35
  Bounds: [-3,889.44, 8,971.98]
  Outliers found: 57

TOTAL_AMOUNT:
  Q1: 16,564.75, Q3: 39,525.00, IQR: 22,960.25
  Bounds: [-17,875.62, 73,965.38]
  Outliers found: 58


### 4.5 Detect Outliers - Z-Score Method

In [37]:
# =============================================================================
# OUTLIER DETECTION - Z-SCORE METHOD
# =============================================================================

def detect_outliers_zscore(data, column, threshold=3):
    """
    Detect outliers using the Z-score method.
    
    Outliers are values with |Z-score| > threshold (typically 3).
    
    Parameters:
        data: DataFrame
        column: Column name to analyze
        threshold: Z-score threshold (default=3)
    
    Returns:
        Dictionary with outlier statistics
    """
    mean_val = data[column].mean()
    std_val = data[column].std()
    
    z_scores = np.abs((data[column] - mean_val) / std_val)
    outliers_mask = z_scores > threshold
    outliers = data[outliers_mask]
    
    return {
        'column': column,
        'mean': mean_val,
        'std': std_val,
        'threshold': threshold,
        'n_outliers': len(outliers),
        'outlier_indices': outliers.index.tolist(),
        'outlier_values': outliers[column].tolist(),
        'z_scores': z_scores[outliers_mask].tolist()
    }

print("\n" + "=" * 60)
print("OUTLIER DETECTION - Z-SCORE METHOD (threshold=3)")
print("=" * 60)

zscore_results = []

for col in numerical_cols:
    result = detect_outliers_zscore(df_clean, col)
    zscore_results.append(result)
    
    if result['n_outliers'] > 0:
        print(f"\n{col.upper()}:")
        print(f"  Mean: {result['mean']:,.2f}, Std: {result['std']:,.2f}")
        print(f"  Outliers found: {result['n_outliers']}")
        if result['n_outliers'] <= 10:
            print(f"  Values: {result['outlier_values']}")
            print(f"  Z-scores: {[f'{z:.2f}' for z in result['z_scores']]}")


OUTLIER DETECTION - Z-SCORE METHOD (threshold=3)

QUANTITY:
  Mean: 1.95, Std: 1.72
  Outliers found: 1
  Values: [50]
  Z-scores: ['27.95']

UNIT_PRICE:
  Mean: 15,364.79, Std: 11,220.78
  Outliers found: 1
  Values: [299900]
  Z-scores: ['25.36']

SUBTOTAL:
  Mean: 29,108.78, Std: 24,177.43
  Outliers found: 20

PLATFORM_FEE:
  Mean: 3,120.51, Std: 2,999.54
  Outliers found: 25

TOTAL_AMOUNT:
  Mean: 31,688.89, Std: 23,050.42
  Outliers found: 20


### 4.6 Compare IQR vs Z-Score Results

In [38]:
# =============================================================================
# COMPARISON: IQR VS Z-SCORE
# =============================================================================

print("=" * 60)
print("COMPARISON: IQR VS Z-SCORE METHODS")
print("=" * 60)

comparison_data = []
for iqr_res, zscore_res in zip(outlier_results, zscore_results):
    comparison_data.append({
        'Column': iqr_res['column'],
        'IQR Outliers': iqr_res['n_outliers'],
        'Z-Score Outliers': zscore_res['n_outliers']
    })

df_comparison = pd.DataFrame(comparison_data)
print("\n")
display(df_comparison)

print("\n--- Interpretation ---")
print("IQR method is more conservative and detects more outliers in skewed distributions.")
print("Z-Score method assumes normal distribution and is more lenient.")
print("For business data with potential extreme values, IQR is often preferred.")

COMPARISON: IQR VS Z-SCORE METHODS




,Column,IQR Outliers,Z-Score Outliers
0,quantity,1,1
1,unit_price,10,1
2,subtotal,52,20
3,shipping_cost,0,0
4,platform_fee,57,25
5,total_amount,58,20



--- Interpretation ---
IQR method is more conservative and detects more outliers in skewed distributions.
Z-Score method assumes normal distribution and is more lenient.
For business data with potential extreme values, IQR is often preferred.


### 4.7 Treat Outliers

In [39]:
# =============================================================================
# OUTLIER TREATMENT - BUSINESS DECISION
# =============================================================================

print("=" * 60)
print("OUTLIER TREATMENT STRATEGY")
print("=" * 60)

# Business rules for outlier treatment:
# 1. unit_price > 100,000 CLP: Likely data entry error → cap at 99th percentile
# 2. quantity > 20: Unusual but possible (bulk order) → keep but flag
# 3. Negative values: Invalid → set to 0 or remove

df_treated = df_clean.copy()

# Convert numeric columns to float for calculations
numeric_cols = ['unit_price', 'subtotal', 'shipping_cost', 'platform_fee', 'total_amount']
df_treated[numeric_cols] = df_treated[numeric_cols].astype(float)

# 1. Cap extreme unit prices
price_cap = df_treated['unit_price'].quantile(0.99)
extreme_prices = df_treated['unit_price'] > 100000
print(f"\n1. Unit prices > 100,000 CLP: {extreme_prices.sum()} found")
if extreme_prices.sum() > 0:
    print(f"   Capping at 99th percentile: {price_cap:,.0f} CLP")
    df_treated.loc[extreme_prices, 'unit_price'] = price_cap
    # Recalculate dependent columns
    df_treated.loc[extreme_prices, 'subtotal'] = df_treated.loc[extreme_prices, 'unit_price'] * df_treated.loc[extreme_prices, 'quantity']
    df_treated.loc[extreme_prices, 'total_amount'] = df_treated.loc[extreme_prices, 'subtotal'] + df_treated.loc[extreme_prices, 'shipping_cost']

# 2. Flag bulk orders (quantity > 20)
bulk_orders = df_treated['quantity'] > 20
print(f"\n2. Bulk orders (quantity > 20): {bulk_orders.sum()} found")
df_treated['is_bulk_order'] = bulk_orders
if bulk_orders.sum() > 0:
    print(f"   Flagged as bulk orders (not removed)")

# 3. Check for negative values
negative_amounts = (df_treated[numerical_cols] < 0).any(axis=1)
print(f"\n3. Rows with negative values: {negative_amounts.sum()} found")
if negative_amounts.sum() > 0:
    print(f"   Removing invalid rows...")
    df_treated = df_treated[~negative_amounts]

print(f"\nFinal dataset size: {len(df_treated)} rows")

OUTLIER TREATMENT STRATEGY

1. Unit prices > 100,000 CLP: 1 found
   Capping at 99th percentile: 33,852 CLP

2. Bulk orders (quantity > 20): 1 found
   Flagged as bulk orders (not removed)

3. Rows with negative values: 0 found

Final dataset size: 1192 rows


### 4.8 Document Cleaning Decisions

In [40]:
# =============================================================================
# DATA QUALITY REPORT
# =============================================================================

print("=" * 60)
print("DATA QUALITY REPORT")
print("=" * 60)

print("\n--- Before Cleaning ---")
print(f"Total rows: {len(df)}")
print(f"Rows with nulls: { df_treated.isnull().any(axis=1).sum()}")

print("\n--- After Cleaning ---")
print(f"Total rows: {len(df_treated)}")
print(f"Rows with nulls: {df_treated.isnull().any(axis=1).sum()}")
print(f"Rows removed: {len(df) - len(df_treated)}")

print("\n--- Cleaning Actions Taken ---")
print("1. Missing values: Imputed with median (numerical) or mode (categorical)")
print("2. Extreme unit prices (>100k): Capped at 99th percentile")
print("3. Bulk orders (qty>20): Flagged but retained")
print("4. Negative values: Removed")

print("\n--- Final Data Statistics ---")
display(df_treated.describe())

DATA QUALITY REPORT

--- Before Cleaning ---
Total rows: 1192
Rows with nulls: 0

--- After Cleaning ---
Total rows: 1192
Rows with nulls: 0
Rows removed: 0

--- Cleaning Actions Taken ---
1. Missing values: Imputed with median (numerical) or mode (categorical)
2. Extreme unit prices (>100k): Capped at 99th percentile
3. Bulk orders (qty>20): Flagged but retained
4. Negative values: Removed

--- Final Data Statistics ---


,customer_id,transaction_date,quantity,unit_price,subtotal,shipping_cost,platform_fee,total_amount
count,1192.000000,1192,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000
mean,170.283557,2023-12-10 08:26:10.469798,1.952181,15141.591032,28913.979044,2746.644295,3120.508221,31498.710587
min,1.000000,2023-01-01 00:00:00,1.000000,3010.000000,3039.000000,0.000000,149.700000,4490.000000
25%,44.750000,2023-06-06 18:00:00,1.000000,10600.000000,12990.000000,0.000000,933.595000,16564.750000
50%,98.000000,2023-11-16 12:00:00,2.000000,13712.500000,22784.500000,3500.000000,1973.400000,26333.500000
75%,294.000000,2024-06-06 06:00:00,3.000000,19990.000000,39033.500000,3500.000000,4148.950000,39525.000000
max,500.000000,2025-01-30 00:00:00,50.000000,34809.000000,249500.000000,5500.000000,17087.720000,249500.000000
std,151.641979,NaN,1.719224,7626.682639,22895.252444,2063.941469,2999.539522,21733.205422


### 4.9 Save Cleaned Data

In [41]:
# =============================================================================
# SAVE CLEANED DATAFRAME
# =============================================================================

# Save to raw folder (still part of the cleaning process)
df_treated.to_csv('../data/raw/transactions_cleaned.csv', index=False)

print("Cleaned data saved:")
print(f"  - transactions_cleaned.csv ({len(df_treated)} rows)")

Cleaned data saved:
  - transactions_cleaned.csv (1192 rows)


### 4.10 Lesson 4 Summary

**What we accomplished:**

1. ✅ Identified null values using `.isnull()` and created a summary report
2. ✅ Applied imputation strategies: median for numerical, mode for categorical
3. ✅ Detected outliers using IQR method with bounds [Q1-1.5*IQR, Q3+1.5*IQR]
4. ✅ Detected outliers using Z-score method with threshold=3
5. ✅ Compared both methods and documented differences
6. ✅ Applied business rules for outlier treatment (cap, flag, remove)
7. ✅ Created a data quality report documenting all decisions
8. ✅ Saved cleaned dataset

**Key Pandas/NumPy methods used:**

- `.isnull()`, `.isna()` - Identify nulls
- `.fillna()` - Impute missing values
- `.quantile()` - Calculate percentiles for IQR
- `.mean()`, `.std()` - For Z-score calculation
- Boolean indexing for filtering outliers

**Business decisions documented:**

- Extreme prices (>100k CLP): Likely errors → capped
- Bulk orders (qty>20): Valid but unusual → flagged
- Negative values: Invalid → removed

**Impact on data quality:**

- Null values eliminated: 100%
- Extreme values corrected: preserves data while limiting distortion
- New feature created: `is_bulk_order` flag

**Next step:** Lesson 5 - Apply Data Wrangling techniques for advanced transformation.

---

## Lesson 5: Data Wrangling

**Objective:** Apply advanced data transformation techniques including duplicate removal, data type conversion, custom functions, and feature engineering.

### Project Evolution - NPS Discovery

> *"During data preparation, exploratory analysis revealed that customer feedback could be consolidated into an NPS metric, which led us to refine the business objective."*

This lesson incorporates NPS survey data discovered during the project, demonstrating how real-world data science projects evolve iteratively.

In [42]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print("Libraries loaded for Lesson 5")

Libraries loaded for Lesson 5


### 5.1 Load Cleaned Data

In [43]:
# =============================================================================
# LOAD CLEANED DATA FROM LESSON 4
# =============================================================================

df_treated = pd.read_csv('../data/raw/transactions_cleaned.csv')
df_treated['transaction_date'] = pd.to_datetime(df['transaction_date'])

print(f"Loaded {len(df_treated)} transactions")

# Load customers data
df_customers = pd.read_csv('../data/raw/customers_preliminary.csv')
df_customers['registration_date'] = pd.to_datetime(df_customers['registration_date'])

print(f"Transactions loaded: {len(df)} rows")
print(f"Customers loaded: {len(df_customers)} rows")

Loaded 1192 transactions
Transactions loaded: 1192 rows
Customers loaded: 500 rows


### 5.2 Generate NPS Survey Data (New Discovery)

**Business Context:** During exploratory analysis, we discovered that customer satisfaction surveys existed but were not integrated. This data enables NPS calculation and customer-aware strategies.

In [44]:
# =============================================================================
# GENERATE NPS SURVEY DATA
# =============================================================================
# Simulating discovery of existing survey data

np.random.seed(123)

# Get unique customers who made purchases
active_customers = df_treated['customer_id'].unique()

# Assume 60% of customers responded to NPS survey
n_respondents = int(len(active_customers) * 0.6)
survey_customers = np.random.choice(active_customers, size=n_respondents, replace=False)

# Generate NPS scores (0-10)
# Distribution: ~20% Detractors (0-6), ~30% Passives (7-8), ~50% Promoters (9-10)
nps_scores = np.concatenate([
    np.random.randint(0, 7, size=int(n_respondents * 0.20)),    # Detractors
    np.random.randint(7, 9, size=int(n_respondents * 0.30)),    # Passives
    np.random.randint(9, 11, size=n_respondents - int(n_respondents * 0.20) - int(n_respondents * 0.30))  # Promoters
])
np.random.shuffle(nps_scores)

# Survey dates (within last 6 months)
base_date = datetime(2024, 7, 1)
survey_dates = [base_date + timedelta(days=np.random.randint(0, 180)) for _ in range(n_respondents)]

# Create NPS DataFrame
df_nps = pd.DataFrame({
    'customer_id': survey_customers,
    'nps_score': nps_scores[:n_respondents],
    'survey_date': survey_dates
})

print(f"NPS Survey data generated: {len(df_nps)} responses")
print(f"\nNPS Score distribution:")
print(df_nps['nps_score'].value_counts().sort_index())

NPS Survey data generated: 235 responses

NPS Score distribution:
nps_score
0      7
1      6
2     10
3      7
4      4
5      6
6      7
7     34
8     36
9     63
10    55
Name: count, dtype: int64


### 5.3 NPS Classification

In [45]:
# =============================================================================
# CLASSIFY NPS RESPONDENTS
# =============================================================================

def classify_nps(score):
    """
    Classify customer based on NPS score.
    
    Parameters:
        score: NPS score (0-10)
    
    Returns:
        Classification: Promoter, Passive, or Detractor
    """
    if score >= 9:
        return 'Promoter'
    elif score >= 7:
        return 'Passive'
    else:
        return 'Detractor'

# Apply classification
df_nps['nps_category'] = df_nps['nps_score'].apply(classify_nps)

# Calculate NPS
nps_counts = df_nps['nps_category'].value_counts()
promoters_pct = nps_counts.get('Promoter', 0) / len(df_nps) * 100
detractors_pct = nps_counts.get('Detractor', 0) / len(df_nps) * 100
nps_score_final = promoters_pct - detractors_pct

print("=" * 60)
print("NPS CALCULATION")
print("=" * 60)
print(f"\nCategory distribution:")
print(nps_counts)
print(f"\nPromoters: {promoters_pct:.1f}%")
print(f"Detractors: {detractors_pct:.1f}%")
print(f"\n>>> NPS Score: {nps_score_final:.1f} <<<")

NPS CALCULATION

Category distribution:
nps_category
Promoter     118
Passive       70
Detractor     47
Name: count, dtype: int64

Promoters: 50.2%
Detractors: 20.0%

>>> NPS Score: 30.2 <<<


### 5.4 Merge NPS with Customer Data

In [46]:
# =============================================================================
# MERGE NPS DATA WITH CUSTOMERS
# =============================================================================

# Merge NPS with customers
df_customers_enriched = df_customers.merge(
    df_nps[['customer_id', 'nps_score', 'nps_category']],
    on='customer_id',
    how='left'
)

# Fill missing NPS (customers who didn't respond)
df_customers_enriched['nps_category'] = df_customers_enriched['nps_category'].fillna('No Response')

print("Customers enriched with NPS data")
print(f"\nNPS Response status:")
print(df_customers_enriched['nps_category'].value_counts())

Customers enriched with NPS data

NPS Response status:
nps_category
No Response    265
Promoter       118
Passive         70
Detractor       47
Name: count, dtype: int64


### 5.5 Data Wrangling - Duplicates

In [47]:
# =============================================================================
# CHECK AND REMOVE DUPLICATES
# =============================================================================

print("=" * 60)
print("DUPLICATE ANALYSIS")
print("=" * 60)

# Check for exact duplicates
exact_duplicates =  df_treated.duplicated().sum()
print(f"\nExact duplicate rows: {exact_duplicates}")

# Check for duplicate transaction IDs
duplicate_ids = df['transaction_id'].duplicated().sum()
print(f"Duplicate transaction IDs: {duplicate_ids}")

# Remove duplicates if any
if exact_duplicates > 0:
    df =  df_treated.drop_duplicates()
    print(f"\nRemoved {exact_duplicates} duplicate rows")
else:
    print("\nNo duplicates to remove")

print(f"\nFinal transaction count: {len(df)}")

DUPLICATE ANALYSIS

Exact duplicate rows: 0
Duplicate transaction IDs: 0

No duplicates to remove

Final transaction count: 1192


### 5.6 Data Wrangling - Type Conversion

In [48]:
# =============================================================================
# DATA TYPE OPTIMIZATION
# =============================================================================

print("=" * 60)
print("DATA TYPE OPTIMIZATION")
print("=" * 60)

print("\nBefore optimization:")
print(df_treated.dtypes)

# Convert categorical columns
categorical_cols = ['platform', 'product']
for col in categorical_cols:
    df_treated[col] = df_treated[col].astype('category')

# Convert boolean
df_treated['is_bulk_order'] = df_treated['is_bulk_order'].astype(bool)

print("\nAfter optimization:")
print(df_treated.dtypes)

# Memory savings
print(f"\nMemory usage: {df_treated.memory_usage(deep=True).sum() / 1024:.1f} KB")

DATA TYPE OPTIMIZATION

Before optimization:
transaction_id                 str
customer_id                  int64
transaction_date    datetime64[us]
platform                       str
product                        str
quantity                     int64
unit_price                 float64
subtotal                   float64
shipping_cost              float64
platform_fee               float64
total_amount               float64
is_bulk_order                 bool
dtype: object

After optimization:
transaction_id                 str
customer_id                  int64
transaction_date    datetime64[us]
platform                  category
product                   category
quantity                     int64
unit_price                 float64
subtotal                   float64
shipping_cost              float64
platform_fee               float64
total_amount               float64
is_bulk_order                 bool
dtype: object

Memory usage: 141.0 KB


### 5.7 Feature Engineering - Time Features

In [49]:
# =============================================================================
# FEATURE ENGINEERING - TIME-BASED FEATURES
# =============================================================================

print("=" * 60)
print("FEATURE ENGINEERING - TIME FEATURES")
print("=" * 60)

# Extract time components
df_treated['year'] = df_treated['transaction_date'].dt.year
df_treated['month'] = df_treated['transaction_date'].dt.month
df_treated['quarter'] = df_treated['transaction_date'].dt.quarter
df_treated['day_of_week'] = df_treated['transaction_date'].dt.dayofweek
df_treated['is_weekend'] = df_treated['day_of_week'].isin([5, 6])

# Season (Chilean seasons - Southern Hemisphere)
def get_season(month):
    """Get Chilean season based on month."""
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Fall'
    elif month in [6, 7, 8]:
        return 'Winter'
    else:
        return 'Spring'

df_treated['season'] = df_treated['month'].apply(get_season)

print("\nNew time features created:")
print(df_treated[['transaction_date', 'year', 'month', 'quarter', 'day_of_week', 'is_weekend', 'season']].head(10))

FEATURE ENGINEERING - TIME FEATURES

New time features created:
  transaction_date  year  month  quarter  day_of_week  is_weekend  season
0       2023-01-01  2023      1        1            6        True  Summer
1       2023-01-01  2023      1        1            6        True  Summer
2       2023-01-01  2023      1        1            6        True  Summer
3       2023-01-01  2023      1        1            6        True  Summer
4       2023-01-03  2023      1        1            1       False  Summer
5       2023-01-04  2023      1        1            2       False  Summer
6       2023-01-04  2023      1        1            2       False  Summer
7       2023-01-05  2023      1        1            3       False  Summer
8       2023-01-07  2023      1        1            5        True  Summer
9       2023-01-07  2023      1        1            5        True  Summer


### 5.8 Feature Engineering - Customer Metrics

In [50]:
# =============================================================================
# FEATURE ENGINEERING - CUSTOMER METRICS
# =============================================================================

print("=" * 60)
print("FEATURE ENGINEERING - CUSTOMER METRICS")
print("=" * 60)

# Calculate customer-level metrics
customer_metrics = df_treated.groupby('customer_id').agg({
    'transaction_id': 'count',
    'total_amount': ['sum', 'mean'],
    'transaction_date': ['min', 'max']
}).reset_index()

# Flatten column names
customer_metrics.columns = [
    'customer_id', 'total_transactions', 'total_revenue', 
    'avg_ticket', 'first_purchase', 'last_purchase'
]

# Calculate days since last purchase (for churn analysis)
reference_date = df_treated['transaction_date'].max()
customer_metrics['days_since_last_purchase'] = (
    reference_date - customer_metrics['last_purchase']
).dt.days

# Calculate customer tenure
customer_metrics['tenure_days'] = (
    customer_metrics['last_purchase'] - customer_metrics['first_purchase']
).dt.days

print(f"\nCustomer metrics calculated for {len(customer_metrics)} customers")
customer_metrics.head(10)

print(f"reference_date usado: {df_treated['transaction_date'].max()}")
print(f"last_purchase max: {customer_metrics['last_purchase'].max()}")
print(f"Diferencia en días: {(df_treated['transaction_date'].max() - customer_metrics['last_purchase'].max()).days}")

FEATURE ENGINEERING - CUSTOMER METRICS

Customer metrics calculated for 392 customers
reference_date usado: 2025-01-30 00:00:00
last_purchase max: 2025-01-30 00:00:00
Diferencia en días: 0


### 5.9 Retargeting Segments

In [51]:
# =============================================================================
# RETARGETING SEGMENTS
# =============================================================================

print("=" * 60)
print("RETARGETING SEGMENTS")
print("=" * 60)

# Define segment thresholds
DORMANT_DAYS = 180    # 6 months
AT_RISK_DAYS = 90     # 3 months

def assign_retargeting_segment(row):
    """
    Assign retargeting segment based on purchase recency.
    
    Segments:
    - Active: Purchased within 60 days
    - At Risk: 60-90 days since purchase
    - Dormant: 90+ days since purchase
    """
    days = row['days_since_last_purchase']
    if days <= AT_RISK_DAYS:
        return 'Active'
    elif days <= DORMANT_DAYS:
        return 'At Risk'
    else:
        return 'Dormant'

customer_metrics['retargeting_segment'] = customer_metrics.apply(assign_retargeting_segment, axis=1)

print("\nRetargeting segment distribution:")
segment_counts = customer_metrics['retargeting_segment'].value_counts()
print(segment_counts)

print("\nPercentage:")
print((segment_counts / len(customer_metrics) * 100).round(1))

RETARGETING SEGMENTS

Retargeting segment distribution:
retargeting_segment
Dormant    207
Active      97
At Risk     88
Name: count, dtype: int64

Percentage:
retargeting_segment
Dormant    52.8
Active     24.7
At Risk    22.4
Name: count, dtype: float64


### 5.10 Merge All Customer Data

In [52]:
# =============================================================================
# MERGE ALL CUSTOMER DATA
# =============================================================================

# Merge customer metrics with enriched customer data
df_customers_final = df_customers_enriched.merge(
    customer_metrics,
    on='customer_id',
    how='left'
)

# Create high-value flag
revenue_threshold = df_customers_final['total_revenue'].quantile(0.75)
df_customers_final['is_high_value'] = df_customers_final['total_revenue'] >= revenue_threshold

# High Value + Dormant = Priority for win-back
df_customers_final['priority_winback'] = (
    (df_customers_final['is_high_value'] == True) & 
    (df_customers_final['retargeting_segment'] == 'Dormant')
)

print("=" * 60)
print("FINAL CUSTOMER DATASET")
print("=" * 60)
print(f"\nTotal customers: {len(df_customers_final)}")
print(f"Columns: {list(df_customers_final.columns)}")

print(f"\nHigh-value customers: {df_customers_final['is_high_value'].sum()}")
print(f"Priority win-back targets: {df_customers_final['priority_winback'].sum()}")

FINAL CUSTOMER DATASET

Total customers: 500
Columns: ['customer_id', 'registration_date', 'region', 'acquisition_channel', 'nps_score', 'nps_category', 'total_transactions', 'total_revenue', 'avg_ticket', 'first_purchase', 'last_purchase', 'days_since_last_purchase', 'tenure_days', 'retargeting_segment', 'is_high_value', 'priority_winback']

High-value customers: 98
Priority win-back targets: 39


### 5.11 Save Wrangled Data

In [53]:
# =============================================================================
# SAVE WRANGLED DATAFRAMES
# =============================================================================

# Save to raw folder
df_treated.to_csv('../data/raw/transactions_wrangled.csv', index=False)
df_customers_final.to_csv('../data/raw/customers_wrangled.csv', index=False)
df_nps.to_csv('../data/raw/nps_surveys.csv', index=False)

print("Wrangled data saved:")
print(f"  - transactions_wrangled.csv ({len(df_treated)} rows)")
print(f"  - customers_wrangled.csv ({len(df_customers_final)} rows)")
print(f"  - nps_surveys.csv ({len(df_nps)} rows)")

Wrangled data saved:
  - transactions_wrangled.csv (1192 rows)
  - customers_wrangled.csv (500 rows)
  - nps_surveys.csv (235 rows)


### 5.12 Lesson 5 Summary

**What we accomplished:**

1. ✅ Discovered and integrated NPS survey data (project evolution)
2. ✅ Classified customers by NPS: Promoter, Passive, Detractor
3. ✅ Calculated overall NPS score
4. ✅ Checked and handled duplicates
5. ✅ Optimized data types (categorical, boolean)
6. ✅ Created time-based features (year, month, quarter, season)
7. ✅ Calculated customer metrics (total transactions, revenue, avg ticket)
8. ✅ Created retargeting segments (Active, At Risk, Dormant)
9. ✅ Identified high-value customers and priority win-back targets

**Key Pandas methods used:**

- `.apply()` - Apply custom functions
- `.merge()` - Join DataFrames
- `.duplicated()`, `.drop_duplicates()` - Handle duplicates
- `.astype()` - Convert data types
- `.dt` accessor - Extract datetime components
- `.groupby().agg()` - Calculate aggregated metrics
- `.fillna()` - Handle missing values

**Business value created:**

- NPS metric enables customer satisfaction tracking
- Retargeting segments enable targeted marketing actions
- High-value + Dormant identification enables prioritized win-back campaigns

**Next step:** Lesson 6 - Apply groupby and pivot operations to calculate final business metrics.

---

## Lesson 6: Grouping and Pivoting

**Objective:** Apply groupby and pivot operations to calculate final business metrics including NPS by segment, CAC, Customer Churn, and Revenue Churn.

### Business Metrics to Calculate

| Metric | Formula | Business Use |
|--------|---------|-------------|
| NPS by Segment | %Promoters - %Detractors | Customer satisfaction tracking |
| CAC | Marketing Spend / New Customers | Marketing efficiency |
| Customer Churn | Lost Customers / Total Active | Retention health |
| Revenue Churn | Lost Revenue / Previous Revenue | Revenue stability |

In [54]:
# Import libraries
import pandas as pd
import numpy as np

print("Libraries loaded for Lesson 6")

Libraries loaded for Lesson 6


### 6.1 Load Wrangled Data

In [55]:
# =============================================================================
# LOAD WRANGLED DATA FROM LESSON 5
# =============================================================================

df_treated  = pd.read_csv('../data/raw/transactions_wrangled.csv')
df_treated['transaction_date'] = pd.to_datetime(df_treated['transaction_date'])

df_customers = pd.read_csv('../data/raw/customers_wrangled.csv')

df_nps = pd.read_csv('../data/raw/nps_surveys.csv')

# Load marketing data
df_marketing = pd.read_csv('../data/raw/marketing_metrics.csv')

print(f"Transactions: {len(df_treated )} rows")
print(f"Customers: {len(df_customers)} rows")
print(f"NPS Surveys: {len(df_nps)} rows")
print(f"Marketing: {len(df_marketing)} rows")


Transactions: 1192 rows
Customers: 500 rows
NPS Surveys: 235 rows
Marketing: 24 rows


### 6.2 Revenue Analysis with groupby()

In [56]:
# =============================================================================
# REVENUE ANALYSIS BY DIMENSIONS
# =============================================================================

print("=" * 60)
print("REVENUE ANALYSIS")
print("=" * 60)

# Revenue by platform
revenue_by_platform = df_treated.groupby('platform').agg({
    'total_amount': ['sum', 'mean', 'count']
}).round(0)
revenue_by_platform.columns = ['Total Revenue', 'Avg Ticket', 'Transactions']

print("\n--- Revenue by Platform ---")
print(revenue_by_platform)

# Revenue by product
revenue_by_product = df_treated.groupby('product').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'quantity': 'sum'
}).round(0)
revenue_by_product.columns = ['Total Revenue', 'Avg Ticket', 'Transactions', 'Units Sold']
revenue_by_product = revenue_by_product.sort_values('Total Revenue', ascending=False)

print("\n--- Revenue by Product ---")
print(revenue_by_product)

# Revenue by season
revenue_by_season = df_treated.groupby('season').agg({
    'total_amount': ['sum', 'mean', 'count']
}).round(0)
revenue_by_season.columns = ['Total Revenue', 'Avg Ticket', 'Transactions']

print("\n--- Revenue by Season ---")
print(revenue_by_season)

REVENUE ANALYSIS

--- Revenue by Platform ---
              Total Revenue  Avg Ticket  Transactions
platform                                             
mercadolibre     29104160.0     32628.0           892
shopify           8442303.0     28141.0           300

--- Revenue by Product ---
         Total Revenue  Avg Ticket  Transactions  Units Sold
product                                                     
jackets     10608062.0     48218.0           220         407
pajamas      6653822.0     38022.0           175         338
shorts       6110060.0     30398.0           201         400
tshirts      5533120.0     27257.0           203         390
towels       5190358.0     28056.0           185         391
socks        3451041.0     16592.0           208         401

--- Revenue by Season ---
        Total Revenue  Avg Ticket  Transactions
season                                         
Fall       10051154.0     32955.0           305
Spring      8476145.0     30164.0           281
Sum

### 6.3 NPS by Segment

In [57]:
# =============================================================================
# NPS ANALYSIS BY SEGMENT
# =============================================================================

print("=" * 60)
print("NPS ANALYSIS BY SEGMENT")
print("=" * 60)

# Filter customers with NPS response
df_with_nps = df_customers[df_customers['nps_category'] != 'No Response'].copy()

# NPS by acquisition channel
def calculate_nps(group):
    """Calculate NPS score for a group."""
    total = len(group)
    promoters = (group['nps_category'] == 'Promoter').sum() / total * 100
    detractors = (group['nps_category'] == 'Detractor').sum() / total * 100
    return promoters - detractors

# NPS by channel
nps_by_channel = df_with_nps.groupby('acquisition_channel').apply(calculate_nps).round(1)
nps_by_channel = nps_by_channel.sort_values(ascending=False)

print("\n--- NPS by Acquisition Channel ---")
print(nps_by_channel)

# NPS by region
nps_by_region = df_with_nps.groupby('region').apply(calculate_nps).round(1)
nps_by_region = nps_by_region.sort_values(ascending=False)

print("\n--- NPS by Region ---")
print(nps_by_region)

# NPS by retargeting segment
nps_by_retargeting = df_with_nps.groupby('retargeting_segment').apply(calculate_nps).round(1)

print("\n--- NPS by Retargeting Segment ---")
print(nps_by_retargeting)

NPS ANALYSIS BY SEGMENT

--- NPS by Acquisition Channel ---
acquisition_channel
google_ads          40.4
organic             39.5
instagram_ads       28.6
mercadolibre_ads    28.6
facebook_ads        11.4
dtype: float64

--- NPS by Region ---
region
OHiggins         54.5
Maule            53.3
Araucania        44.4
Biobio           41.4
Los Lagos        30.0
Metropolitana    25.0
Valparaiso       22.2
Coquimbo        -10.0
dtype: float64

--- NPS by Retargeting Segment ---
retargeting_segment
Active     27.1
At Risk    17.0
Dormant    37.4
dtype: float64


### 6.4 CAC Calculation

In [58]:
# =============================================================================
# CUSTOMER ACQUISITION COST (CAC)
# =============================================================================
# Note: CAC was estimated using paid media spend only.
# A production implementation would include sales team costs,
# marketing tools, and attributed overhead.

print("=" * 60)
print("CUSTOMER ACQUISITION COST (CAC)")
print("=" * 60)

# Marketing spend by channel
marketing_spend = df_marketing.groupby('Channel')['Spend (CLP)'].sum()

# Map marketing channels to acquisition channels
channel_mapping = {
    'Facebook': 'facebook_ads',
    'Instagram': 'instagram_ads',
    'Google': 'google_ads',
    'MercadoLibre': 'mercadolibre_ads'
}

# Count new customers by channel
customers_by_channel = df_customers.groupby('acquisition_channel').size()

# Calculate CAC
cac_data = []
for mkt_channel, acq_channel in channel_mapping.items():
    spend = marketing_spend.get(mkt_channel, 0)
    customers = customers_by_channel.get(acq_channel, 0)
    cac = spend / customers if customers > 0 else 0
    cac_data.append({
        'Channel': acq_channel,
        'Spend (CLP)': spend,
        'New Customers': customers,
        'CAC (CLP)': round(cac, 0)
    })

df_cac = pd.DataFrame(cac_data)
df_cac = df_cac.sort_values('CAC (CLP)')

print("\n--- CAC by Channel ---")
print(df_cac.to_string(index=False))

# Overall CAC
total_spend = df_cac['Spend (CLP)'].sum()
total_customers = df_cac['New Customers'].sum()
overall_cac = total_spend / total_customers if total_customers > 0 else 0

print(f"\n>>> Overall CAC: ${overall_cac:,.0f} CLP <<<")

CUSTOMER ACQUISITION COST (CAC)

--- CAC by Channel ---
         Channel  Spend (CLP)  New Customers  CAC (CLP)
mercadolibre_ads       760000            138     5507.0
      google_ads      2640000            104    25385.0
    facebook_ads      3450000            103    33495.0
   instagram_ads      3110000             74    42027.0

>>> Overall CAC: $23,771 CLP <<<


### 6.5 Churn Analysis

In [59]:
# =============================================================================
# CUSTOMER CHURN ANALYSIS
# =============================================================================

print("=" * 60)
print("CHURN ANALYSIS")
print("=" * 60)

# Customer Churn (based on retargeting segments)
# Dormant = Churned (no purchase in 90+ days)
segment_counts = df_customers['retargeting_segment'].value_counts()

total_customers = len(df_customers)
dormant_customers = segment_counts.get('Dormant', 0)
at_risk_customers = segment_counts.get('At Risk', 0)
active_customers = segment_counts.get('Active', 0)

customer_churn_rate = dormant_customers / total_customers * 100
at_risk_rate = at_risk_customers / total_customers * 100

print("\n--- Customer Churn Metrics ---")
print(f"Total Customers: {total_customers}")
print(f"Active: {active_customers} ({active_customers/total_customers*100:.1f}%)")
print(f"At Risk: {at_risk_customers} ({at_risk_rate:.1f}%)")
print(f"Dormant (Churned): {dormant_customers} ({customer_churn_rate:.1f}%)")

print(f"\n>>> Customer Churn Rate: {customer_churn_rate:.1f}% <<<")

CHURN ANALYSIS

--- Customer Churn Metrics ---
Total Customers: 500
Active: 97 (19.4%)
At Risk: 88 (17.6%)
Dormant (Churned): 207 (41.4%)

>>> Customer Churn Rate: 41.4% <<<


### 6.6 Revenue Churn

In [60]:
# 6.6 =============================================================================
# REVENUE CHURN ANALYSIS
# =============================================================================
print("=" * 60)
print("REVENUE CHURN")
print("=" * 60)

# Revenue by quarter
df_treated['year_quarter'] = df_treated['year'].astype(str) + '-Q' + df_treated['quarter'].astype(str)
revenue_by_quarter = df_treated.groupby('year_quarter')['total_amount'].sum().sort_index()

print("\n--- Revenue by Quarter ---")
print(revenue_by_quarter)

# Calculate quarter-over-quarter revenue churn
revenue_changes = []
quarters = revenue_by_quarter.index.tolist()

for i in range(1, len(quarters)):
    prev_rev = revenue_by_quarter[quarters[i-1]]
    curr_rev = revenue_by_quarter[quarters[i]]
    change = curr_rev - prev_rev
    change_pct = (change / prev_rev * 100) if prev_rev > 0 else 0
    
    revenue_changes.append({
        'Period': f"{quarters[i-1]} → {quarters[i]}",
        'Previous': prev_rev,
        'Current': curr_rev,
        'Change': change,
        'Change %': round(change_pct, 1)
    })

df_revenue_churn = pd.DataFrame(revenue_changes)
print("\n--- Quarter-over-Quarter Revenue Change ---")
print(df_revenue_churn.to_string(index=False))

# Identify negative growth quarters (revenue churn)
negative_quarters = df_revenue_churn[df_revenue_churn['Change'] < 0]
if len(negative_quarters) > 0:
    avg_revenue_churn = negative_quarters['Change %'].mean()
    print(f"\n>>> Average Revenue Churn (negative quarters): {avg_revenue_churn:.1f}% <<<")
else:
    print("\n>>> No revenue churn detected (all quarters positive) <<<")

REVENUE CHURN

--- Revenue by Quarter ---
year_quarter
2023-Q1    5381459.00
2023-Q2    5677501.00
2023-Q3    5774546.00
2023-Q4    4853663.00
2024-Q1    3657372.02
2024-Q2    4286889.00
2024-Q3    3718966.00
2024-Q4    3611467.00
2025-Q1     584600.00
Name: total_amount, dtype: float64

--- Quarter-over-Quarter Revenue Change ---
           Period   Previous    Current      Change  Change %
2023-Q1 → 2023-Q2 5381459.00 5677501.00   296042.00       5.5
2023-Q2 → 2023-Q3 5677501.00 5774546.00    97045.00       1.7
2023-Q3 → 2023-Q4 5774546.00 4853663.00  -920883.00     -15.9
2023-Q4 → 2024-Q1 4853663.00 3657372.02 -1196290.98     -24.6
2024-Q1 → 2024-Q2 3657372.02 4286889.00   629516.98      17.2
2024-Q2 → 2024-Q3 4286889.00 3718966.00  -567923.00     -13.2
2024-Q3 → 2024-Q4 3718966.00 3611467.00  -107499.00      -2.9
2024-Q4 → 2025-Q1 3611467.00  584600.00 -3026867.00     -83.8

>>> Average Revenue Churn (negative quarters): -28.1% <<<


### 6.7 Pivot Tables

In [61]:
# =============================================================================
# PIVOT TABLES
# =============================================================================
print("=" * 60)
print("PIVOT TABLE ANALYSIS")
print("=" * 60)

# Revenue by Platform x Season
pivot_platform_season = pd.pivot_table(
    df_treated,
    values='total_amount',
    index='platform',
    columns='season',
    aggfunc='sum',
    fill_value=0
).round(0)

print("\n--- Revenue: Platform x Season ---")
print(pivot_platform_season)

# Transactions by Product x Platform
pivot_product_platform = pd.pivot_table(
    df_treated,
    values='transaction_id',
    index='product',
    columns='platform',
    aggfunc='count',
    fill_value=0
)

print("\n--- Transactions: Product x Platform ---")
print(pivot_product_platform)

# Average Ticket by Product x Season
pivot_ticket = pd.pivot_table(
    df_treated,
    values='total_amount',
    index='product',
    columns='season',
    aggfunc='mean',
    fill_value=0
).round(0)

print("\n--- Average Ticket: Product x Season ---")
print(pivot_ticket)

PIVOT TABLE ANALYSIS

--- Revenue: Platform x Season ---
season             Fall     Spring     Summer     Winter
platform                                                
mercadolibre  7689751.0  6632765.0  6783224.0  7998420.0
shopify       2361403.0  1843380.0  2516450.0  1721070.0

--- Transactions: Product x Platform ---
platform  mercadolibre  shopify
product                        
jackets            162       58
pajamas            135       40
shorts             138       63
socks              155       53
towels             141       44
tshirts            161       42

--- Average Ticket: Product x Season ---
season      Fall   Spring   Summer   Winter
product                                    
jackets  49953.0  46860.0  46470.0  49359.0
pajamas  41588.0  30959.0  34898.0  42882.0
shorts   29068.0  30313.0  33506.0  28860.0
socks    16371.0  17103.0  16434.0  16566.0
towels   32325.0  24343.0  28065.0  27202.0
tshirts  25112.0  28109.0  27325.0  28236.0


### 6.8 KPI Summary Dashboard

In [62]:
# =============================================================================
# KPI SUMMARY DASHBOARD
# =============================================================================

print("=" * 60)
print("KPI SUMMARY DASHBOARD - PEQUESHOP")
print("=" * 60)

# Calculate final metrics
total_revenue = df['total_amount'].sum()
total_transactions = len(df)
avg_ticket = df['total_amount'].mean()
total_customers_active = len(df_customers)

# NPS (from earlier calculation)
nps_respondents = df_customers[df_customers['nps_category'] != 'No Response']
promoters_pct = (nps_respondents['nps_category'] == 'Promoter').sum() / len(nps_respondents) * 100
detractors_pct = (nps_respondents['nps_category'] == 'Detractor').sum() / len(nps_respondents) * 100
nps_score = promoters_pct - detractors_pct

print("\n📊 CUSTOMER HEALTH")
print(f"   NPS Score: {nps_score:.1f}")
print(f"   Customer Churn Rate: {customer_churn_rate:.1f}%")
print(f"   At-Risk Customers: {at_risk_rate:.1f}%")

print("\n💰 REVENUE")
print(f"   Total Revenue: ${total_revenue:,.0f} CLP")
print(f"   Total Transactions: {total_transactions:,}")
print(f"   Average Ticket: ${avg_ticket:,.0f} CLP")

print("\n🎯 ACQUISITION")
print(f"   Total Customers: {total_customers_active}")
print(f"   Overall CAC: ${overall_cac:,.0f} CLP")

print("\n🔄 RETARGETING SEGMENTS")
for segment in ['Active', 'At Risk', 'Dormant']:
    count = segment_counts.get(segment, 0)
    pct = count / total_customers * 100
    print(f"   {segment}: {count} ({pct:.1f}%)")

# High-value targets
priority_winback = df_customers['priority_winback'].sum() if 'priority_winback' in df_customers.columns else 0
print(f"\n🎯 Priority Win-back Targets: {priority_winback}")


print(df_customers['retargeting_segment'].value_counts())
print(f"\nDays since last purchase - stats:")
print(df_customers['days_since_last_purchase'].describe())

KPI SUMMARY DASHBOARD - PEQUESHOP

📊 CUSTOMER HEALTH
   NPS Score: 30.2
   Customer Churn Rate: 41.4%
   At-Risk Customers: 17.6%

💰 REVENUE
   Total Revenue: $37,773,160 CLP
   Total Transactions: 1,192
   Average Ticket: $31,689 CLP

🎯 ACQUISITION
   Total Customers: 500
   Overall CAC: $23,771 CLP

🔄 RETARGETING SEGMENTS
   Active: 97 (19.4%)
   At Risk: 88 (17.6%)
   Dormant: 207 (41.4%)

🎯 Priority Win-back Targets: 39
retargeting_segment
Dormant    207
Active      97
At Risk     88
Name: count, dtype: int64

Days since last purchase - stats:
count    392.000000
mean     250.127551
std      191.620827
min        0.000000
25%       91.000000
50%      201.000000
75%      377.000000
max      760.000000
Name: days_since_last_purchase, dtype: float64


### 6.9 Export Final Datasets

In [63]:
# =============================================================================
# EXPORT FINAL DATASETS
# =============================================================================

print("=" * 60)
print("EXPORTING FINAL DATASETS")
print("=" * 60)

# Export to processed folder (final clean data)
df_treated.to_csv('../data/processed/transactions_final.csv', index=False)
df_customers.to_csv('../data/processed/customers_final.csv', index=False)

# Export to Excel for business stakeholders
with pd.ExcelWriter('../data/processed/pequeshop_analytics.xlsx', engine='openpyxl') as writer:
    df_treated.to_excel(writer, sheet_name='Transactions', index=False)
    df_customers.to_excel(writer, sheet_name='Customers', index=False)
    df_nps.to_excel(writer, sheet_name='NPS_Surveys', index=False)
    df_cac.to_excel(writer, sheet_name='CAC_Analysis', index=False)
    revenue_by_product.to_excel(writer, sheet_name='Revenue_by_Product')
    pivot_platform_season.to_excel(writer, sheet_name='Revenue_Platform_Season')

print("\n✅ CSV Files exported to data/processed/:")
print(f"   - transactions_final.csv ({len(df_treated)} rows)")
print(f"   - customers_final.csv ({len(df_customers)} rows)")

print("\n✅ Excel workbook exported:")
print(f"   - pequeshop_analytics.xlsx (6 sheets)")

EXPORTING FINAL DATASETS

✅ CSV Files exported to data/processed/:
   - transactions_final.csv (1192 rows)
   - customers_final.csv (500 rows)

✅ Excel workbook exported:
   - pequeshop_analytics.xlsx (6 sheets)


### 6.10 Lesson 6 Summary

**What we accomplished:**

1. ✅ Revenue analysis by platform, product, and season using `groupby()`
2. ✅ NPS calculation by acquisition channel, region, and retargeting segment
3. ✅ CAC calculation by marketing channel
4. ✅ Customer Churn analysis based on purchase recency
5. ✅ Revenue Churn analysis (quarter-over-quarter)
6. ✅ Pivot tables for multi-dimensional analysis
7. ✅ KPI Summary Dashboard
8. ✅ Exported final datasets (CSV + Excel)

**Key Pandas methods used:**

- `.groupby().agg()` - Aggregate by dimensions
- `.groupby().apply()` - Custom aggregation functions
- `pd.pivot_table()` - Multi-dimensional analysis
- `.to_csv()`, `.to_excel()` - Export data
- `pd.ExcelWriter()` - Multi-sheet Excel export

**Business metrics calculated:**

| Metric | Value | Interpretation |
|--------|-------|----------------|
| NPS | Calculated | Customer satisfaction index |
| CAC | By channel | Marketing efficiency |
| Customer Churn | % dormant | Retention health |
| Revenue Churn | QoQ change | Revenue stability |

**Deliverables created:**

- `transactions_final.csv` - Clean transaction data
- `customers_final.csv` - Enriched customer data with NPS and segments
- `pequeshop_analytics.xlsx` - Business-ready Excel workbook

---

## Project Complete! 🎉

### ETL Pipeline Summary

```
EXTRACT (L1-L3)           TRANSFORM (L4-L5)           LOAD (L6)
─────────────────────────────────────────────────────────────────
NumPy generation    →    Missing values      →    CSV exports
CSV loading         →    Outlier treatment   →    Excel workbook
Excel loading       →    NPS integration     →    KPI dashboard
HTML parsing        →    Feature engineering
                    →    Retargeting segments
```

### CRISP-DM Phases Covered

- ✅ **Business Understanding:** Problem definition, KPI framework
- ✅ **Data Understanding:** Exploratory analysis, data profiling
- ✅ **Data Preparation:** Complete ETL pipeline

### Business Value Delivered

1. **Unified data** from 3 sources (MercadoLibre, Shopify, Marketing)
2. **NPS tracking** enables customer satisfaction measurement
3. **Retargeting segments** enable targeted marketing actions
4. **CAC by channel** enables marketing budget optimization
5. **Churn metrics** enable proactive retention strategies

### Next Steps (Future Work)

- Modeling: Predictive models for churn, CLTV
- Evaluation: A/B testing of pricing strategies
- Deployment: Dashboard in Power BI / Streamlit